### 새로운 핵심키워드_v2 기존 데이터와 합치기

In [2]:
import pandas as pd

# 성취기준 데이터 로드
achievement_data = pd.read_excel('merged_final_data.xlsx')  # 성취기준 데이터 파일
keywords_data = pd.read_excel('성취기준.xlsx')  # 핵심키워드_v2 파일

# 성취기준코드를 기준으로 병합
merged_data = pd.merge(achievement_data, keywords_data, on='성취기준코드', how='left')


# 결과 저장
merged_data.to_excel('merged_final_data_new.xlsx', index=False)


### 엑셀파일 => csv 변환

In [7]:
import pandas as pd

# 엑셀 파일 경로
excel_file = 'merged_final_data_new - 복사본.xlsx'  # 엑셀 파일 이름 또는 경로
csv_file = 'merged_final_data_new - 복사본.csv'  # 변환할 CSV 파일 이름

# 엑셀 파일 읽기
df = pd.read_excel(excel_file)

# CSV 파일로 저장
df.to_csv(csv_file, index=False, encoding='utf-8-sig')

print(f"엑셀 파일이 '{csv_file}'로 성공적으로 변환되었습니다!")


엑셀 파일이 'merged_final_data_new - 복사본.csv'로 성공적으로 변환되었습니다!


In [48]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:

# 1. 데이터 로드
file_name = "merged_final_data_new - 복사본.csv"  
data = pd.read_csv(file_name)

# 필수 컬럼 확인
required_columns = [
    "f_mchapter_nm", "f_mchapter_id", "성취기준 내용", "성취기준코드", "핵심키워드_v2",
    "성취수준 A", "성취수준 B", "성취수준 C", "f_schapter_id", "f_schapter_nm"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"필수 컬럼 '{col}'이(가) 데이터에 없습니다.")

# 중복된 텍스트 제거
data = data.drop_duplicates(subset=["f_mchapter_nm", "성취기준 내용", "핵심키워드_v2"])

# 2. 데이터 결합 및 전처리
# 텍스트 정제
data["combined_text"] = (
    data["f_mchapter_nm"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["성취기준 내용"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["핵심키워드_v2"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True)
)
data["tooltip"] = (
    "소단원명: " + data["f_schapter_nm"].fillna("") +
    "<br>성취수준 A: " + data["성취수준 A"].fillna("") +
    "<br>성취수준 B: " + data["성취수준 B"].fillna("") +
    "<br>성취수준 C: " + data["성취수준 C"].fillna("")
)

# 3. 유사도 계산
# Sentence-BERT 유사도
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF 유사도
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["combined_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 최종 유사도
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 4. 그래프 생성
threshold = 0.8
graph = nx.DiGraph()

# 노드 추가
for _, row in data.iterrows():
    level_color = "red" if pd.notna(row["성취수준 A"]) else "orange" if pd.notna(row["성취수준 B"]) else "yellow"
    graph.add_node(
        str(row["f_mchapter_id"]),
        label=row["f_mchapter_nm"],
        tooltip=row["tooltip"],
        color=level_color,
        size=20 if pd.notna(row["성취수준 A"]) else 15 if pd.notna(row["성취수준 B"]) else 10
    )

# 소단원 노드 추가
for _, row in data.iterrows():
    if pd.notna(row["f_schapter_nm"]):
        graph.add_node(
            str(row["f_schapter_id"]),
            label=row["f_schapter_nm"],
            tooltip="소단원",
            color="lightblue",
            size=10
        )
        # 상위 단원과 연결
        graph.add_edge(
            str(row["f_mchapter_id"]),
            str(row["f_schapter_id"]),
            weight=1,
            title="소단원 연결"
        )

# 엣지 추가 (자기 자신 비교 제외)
for i in range(len(data)):
    for j in range(len(data)):
        if i != j and final_sim[i, j] >= threshold:
            graph.add_edge(
                str(data.iloc[i]["f_mchapter_id"]),
                str(data.iloc[j]["f_mchapter_id"]),
                weight=final_sim[i, j],
                title=f"유사도: {final_sim[i, j]:.2f}"
            )

# 5. Pyvis 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -3000,
      "centralGravity": 0.3,
      "springLength": 200,
      "springConstant": 0.05
    },
    "minVelocity": 0.1
  }
}
""")

# HTML 저장
output_file = "new_knowledge_graph_v2.html"
net.write_html(output_file)
print(f"HTML 파일이 '{output_file}'로 저장되었습니다.")


HTML 파일이 'new_knowledge_graph_v2.html'로 저장되었습니다.


### 유사도 기반 + 성취기준 기반 선후행 관계 포함

In [60]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_nm", "f_mchapter_id", "성취기준코드", "성취기준 내용", 
    "성취수준 A", "성취수준 B", "성취수준 C", "f_schapter_id", 
    "f_schapter_nm", "핵심키워드_v2"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data = data.drop_duplicates(subset=["f_mchapter_nm", "성취기준 내용", "핵심키워드_v2"])
data["combined_text"] = (
    data["f_mchapter_nm"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["성취기준 내용"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["핵심키워드_v2"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True)
)
data["tooltip"] = (
    "소단원명: " + data["f_schapter_nm"].fillna("") +
    "<br>성취수준 A: " + data["성취수준 A"].fillna("") +
    "<br>성취수준 B: " + data["성취수준 B"].fillna("") +
    "<br>성취수준 C: " + data["성취수준 C"].fillna("")
)

# Calculate similarities
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(embeddings.cpu(), embeddings.cpu())

# Calculate TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["combined_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim
threshold = 0.75

# Create the graph
graph = nx.DiGraph()

# Add nodes for 성취기준코드 and 단원
for _, row in data.iterrows():
    graph.add_node(
        row["성취기준코드"],
        label=row["f_mchapter_nm"] + " - " + row["성취기준코드"],
        tooltip=row["tooltip"],
        color="red",
        size=20
    )

# Add predefined edges based on 성취기준 선후행 관계
predefined_edges = [
    ("2수01-01", "2수01-02"),
    ("2수01-02", "2수01-03"),
    ("2수01-03", "2수01-04"),
    ("2수01-05", "2수01-06"),
    ("2수01-06", "2수01-07"),
    ("2수01-07", "2수01-08"),
    ("2수01-08", "2수01-09"),
    ("2수01-10", "2수01-11"),
    ("2수02-01", "2수02-02"),
    ("2수03-01", "2수03-02"),
    ("2수03-03", "2수03-04"),
    ("2수03-04", "2수03-05"),
    ("2수03-06", "2수03-07"),
    ("2수03-07", "2수03-08"),
    ("2수03-08", "2수03-09"),
    ("2수03-10", "2수03-11"),
    ("2수03-11", "2수03-12"),
    ("2수03-12", "2수03-13"),
    ("2수04-01", "2수04-02"),
    ("2수04-02", "2수04-03")
]
for edge in predefined_edges:
    graph.add_edge(edge[0], edge[1], weight=1.0, title="선후행 관계")

# Add similarity-based edges (유사도 기반 연결)
for i in range(len(data)):
    for j in range(len(data)):
        if i != j and final_sim[i, j] >= threshold:
            source = data.iloc[i]["성취기준코드"]
            target = data.iloc[j]["성취기준코드"]
            if not graph.has_edge(source, target):
                graph.add_edge(
                    source,
                    target,
                    weight=final_sim[i, j],
                    title=f"유사도: {final_sim[i, j]:.2f}",
                    color="blue",
                    dash=True
                )

# Visualize with PyVis
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Customize physics for better layout
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.4,
      "springLength": 200,
      "springConstant": 0.1
    },
    "minVelocity": 0.5
  }
}
""")

# Save visualization
output_file = "new_knowledge_graph_v3.html"
net.write_html(output_file)
print(f"Integrated graph saved to {output_file}")


Integrated graph saved to new_knowledge_graph_v3.html


In [ ]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_nm", "f_mchapter_id", "성취기준코드", "성취기준 내용", 
    "성취수준 A", "성취수준 B", "성취수준 C", "f_schapter_id", 
    "f_schapter_nm", "핵심키워드_v2"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data = data.drop_duplicates(subset=["f_mchapter_nm", "성취기준 내용", "핵심키워드_v2"])
data["combined_text"] = (
    data["f_mchapter_nm"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["성취기준 내용"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["핵심키워드_v2"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True)
)
data["tooltip"] = (
    "소단원명: " + data["f_schapter_nm"].fillna("") +
    "<br>성취수준 A: " + data["성취수준 A"].fillna("") +
    "<br>성취수준 B: " + data["성취수준 B"].fillna("") +
    "<br>성취수준 C: " + data["성취수준 C"].fillna("")
)

# Calculate similarities
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(embeddings.cpu(), embeddings.cpu())

# Calculate TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["combined_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim
threshold = 0.7

# Create the graph
graph = nx.DiGraph()

# Add nodes for 성취기준코드 and 단원
for _, row in data.iterrows():
    graph.add_node(
        row["성취기준코드"],
        label=row["f_mchapter_nm"] + " - " + row["성취기준코드"],
        tooltip=row["tooltip"],
        color="red",
        size=20
    )

# Add predefined edges based on 성취기준 선후행 관계
predefined_edges = [
    ("2수01-01", "2수01-02"),
    ("2수01-02", "2수01-03"),
    ("2수01-03", "2수01-04"),
    ("2수01-05", "2수01-06"),
    ("2수01-06", "2수01-07"),
    ("2수01-07", "2수01-08"),
    ("2수01-08", "2수01-09"),
    ("2수01-10", "2수01-11"),
    ("2수02-01", "2수02-02"),
    ("2수03-01", "2수03-02"),
    ("2수03-03", "2수03-04"),
    ("2수03-04", "2수03-05"),
    ("2수03-06", "2수03-07"),
    ("2수03-07", "2수03-08"),
    ("2수03-08", "2수03-09"),
    ("2수03-10", "2수03-11"),
    ("2수03-11", "2수03-12"),
    ("2수03-12", "2수03-13"),
    ("2수04-01", "2수04-02"),
    ("2수04-02", "2수04-03")
]
for edge in predefined_edges:
    graph.add_edge(edge[0], edge[1], weight=1.0, title="선후행 관계")

# Add similarity-based edges (유사도 기반 연결)
for i in range(len(data)):
    for j in range(len(data)):
        if i != j and final_sim[i, j] >= threshold:
            source = data.iloc[i]["성취기준코드"]
            target = data.iloc[j]["성취기준코드"]
            if not graph.has_edge(source, target):
                graph.add_edge(
                    source,
                    target,
                    weight=final_sim[i, j],
                    title=f"유사도: {final_sim[i, j]:.2f}",
                    color="blue",
                    dash=True
                )

# Visualize with PyVis
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Customize physics for better layout
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.4,
      "springLength": 200,
      "springConstant": 0.1
    },
    "minVelocity": 0.5
  }
}
""")

# Save visualization
output_file = "new_knowledge_graph_v4.html"
net.write_html(output_file)
print(f"Integrated graph saved to {output_file}")


Integrated graph saved to new_knowledge_graph_v4.html


### v2+v4

In [30]:
import pandas as pd
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_nm", "f_mchapter_id", "성취기준코드", "성취기준 내용", 
    "성취수준 A", "성취수준 B", "성취수준 C", "f_schapter_id", 
    "f_schapter_nm", "핵심키워드_v2"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data = data.drop_duplicates(subset=["f_mchapter_nm", "성취기준 내용", "핵심키워드_v2"])
data["combined_text"] = (
    data["f_mchapter_nm"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["성취기준 내용"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["핵심키워드_v2"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True)
)
data["tooltip"] = (
    "소단원명: " + data["f_schapter_nm"].fillna("") +
    "<br>성취수준 A: " + data["성취수준 A"].fillna("") +
    "<br>성취수준 B: " + data["성취수준 B"].fillna("") +
    "<br>성취수준 C: " + data["성취수준 C"].fillna("")
)

# Calculate similarities
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(embeddings.cpu(), embeddings.cpu()).astype('float64')  # Ensure float64

# Combine similarities
threshold = 0.85  # Threshold for similarity
graph = nx.DiGraph()

# Add nodes for 중단원
for _, row in data.iterrows():
    graph.add_node(
        row["성취기준코드"],
        label=row["f_mchapter_nm"] + " - " + row["성취기준코드"],
        tooltip=row["tooltip"],
        color="green",
        size=20
    )

# Add predefined edges based on 성취기준 선후행 관계
predefined_edges = [
    ("2수01-01", "2수01-02"),
    ("2수01-02", "2수01-03"),
    ("2수01-03", "2수01-04"),
    ("2수01-05", "2수01-06"),
    ("2수01-06", "2수01-07"),
    ("2수01-07", "2수01-08"),
    ("2수01-08", "2수01-09"),
    ("2수01-10", "2수01-11"),
    ("2수02-01", "2수02-02"),
    ("2수03-01", "2수03-02"),
    ("2수03-03", "2수03-04"),
    ("2수03-04", "2수03-05"),
    ("2수03-06", "2수03-07"),
    ("2수03-07", "2수03-08"),
    ("2수03-08", "2수03-09"),
    ("2수03-10", "2수03-11"),
    ("2수03-11", "2수03-12"),
    ("2수03-12", "2수03-13"),
    ("2수04-01", "2수04-02"),
    ("2수04-02", "2수04-03")
]
for edge in predefined_edges:
    graph.add_edge(edge[0], edge[1], weight=1.0, title="선후행 관계", color="blue")

# Add similarity-based edges
for i in range(len(data)):
    for j in range(i + 1, len(data)):  # Only forward connections allowed
        if bert_sim[i, j] >= threshold:
            source = data.iloc[i]["성취기준코드"]
            target = data.iloc[j]["성취기준코드"]
            graph.add_edge(
                source,
                target,
                weight=float(bert_sim[i, j]),  # Ensure float is JSON serializable
                title=f"유사도: {bert_sim[i, j]:.2f}",
                color="red"
            )

# Remove self-loops and duplicated edges
edges_to_remove = []
for u, v in graph.edges():
    if u == v:  # Remove self-loops
        edges_to_remove.append((u, v))
for edge in edges_to_remove:
    graph.remove_edge(*edge)

# Visualize with PyVis
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Customize physics for better layout
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -3000,
      "centralGravity": 0.3,
      "springLength": 150,
      "springConstant": 0.05
    },
    "minVelocity": 0.5
  },
  "nodes": {
    "shape": "dot",
    "size": 20
  },
  "edges": {
    "smooth": {
      "type": "continuous"
    }
  }
}
""")

# Save visualization
output_file = "new_knowledge_graph_v5.html"
net.write_html(output_file)
print(f"Integrated graph saved to {output_file}")


Integrated graph saved to new_knowledge_graph_v5.html


In [31]:
import pandas as pd
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = ["f_mchapter_nm", "성취기준코드", "성취기준 내용", "핵심키워드_v2"]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Combine text for similarity calculation
data["combined_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["핵심키워드_v2"].fillna("")
)

# Sort data by 성취기준코드
data = data.sort_values(by="성취기준코드").reset_index(drop=True)

# Perform similarity-based analysis
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
similarity_matrix = cosine_similarity(embeddings.cpu(), embeddings.cpu())

# Generate 성취기준 흐름 기반 edges
flow_edges = []
for i in range(len(data) - 1):
    flow_edges.append((data.iloc[i]["성취기준코드"], data.iloc[i + 1]["성취기준코드"], "선후행 관계"))

# Generate 유사도 기반 edges
threshold = 0.75  # Set a high threshold for similarity
similarity_edges = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):  # Only forward connections allowed
        if similarity_matrix[i, j] >= threshold:
            similarity_edges.append((data.iloc[i]["성취기준코드"], data.iloc[j]["성취기준코드"], "유사도 기반 연결"))

# Combine edges
all_edges = flow_edges + similarity_edges

# Map 성취기준코드 to 중단원명
code_to_chapter = data.set_index("성취기준코드")["f_mchapter_nm"].to_dict()

# Create the graph
graph = nx.DiGraph()

# Add nodes for 중단원
unique_mchapters = data["f_mchapter_nm"].unique()
for chapter in unique_mchapters:
    graph.add_node(
        chapter,
        label=chapter,
        color="green",
        size=30
    )

# Add edges to the graph
for source, target, edge_type in all_edges:
    source_chapter = code_to_chapter[source]
    target_chapter = code_to_chapter[target]
    if source_chapter != target_chapter:  # Avoid self-loops
        graph.add_edge(
            source_chapter, target_chapter,
            color="blue" if edge_type == "선후행 관계" else "red",
            title=edge_type
        )

# Visualize with PyVis
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Customize physics for better layout
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.4,
      "springLength": 150,
      "springConstant": 0.05
    },
    "minVelocity": 0.5
  }
}
""")

# Save visualization
output_file = "new_knowledge_graph_v6.html"
net.write_html(output_file)
print(f"Optimized graph saved to {output_file}")


Optimized graph saved to new_knowledge_graph_v6.html


### 성취기준 코드 흐름으로만

In [32]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = ["f_mchapter_nm", "성취기준코드"]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess 중단원명
data["f_mchapter_nm"] = (
    data["f_mchapter_nm"]
    .str.strip()  # 공백 제거
    .str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)  # 특수문자 제거
)

# Sort data by 성취기준코드 to establish flow
data = data.sort_values(by="성취기준코드").reset_index(drop=True)

# Map 성취기준코드 to 중단원명
code_to_chapter = data.set_index("성취기준코드")["f_mchapter_nm"].to_dict()

# Generate edges based on 성취기준코드 flow
edges = []
previous_chapter = None
for _, row in data.iterrows():
    current_chapter = row["f_mchapter_nm"]
    if previous_chapter and previous_chapter != current_chapter:
        edges.append((previous_chapter, current_chapter))
    previous_chapter = current_chapter

# Extract unique 중단원명
unique_mchapters = data["f_mchapter_nm"].drop_duplicates().tolist()

# Create the graph
graph = nx.DiGraph()

# Add nodes for 중단원명
for chapter in unique_mchapters:
    graph.add_node(
        chapter,
        label=chapter,
        color="green",
        size=30
    )

# Add edges for 중단원 선후행 관계
for edge in edges:
    graph.add_edge(
        edge[0], edge[1],
        weight=1.0,
        color="blue",
        title="성취기준코드 흐름 기반 중단원 선후행 관계"
    )

# Add edges to ensure all nodes are connected if needed
connected_components = list(nx.weakly_connected_components(graph))
if len(connected_components) > 1:
    for i in range(len(connected_components) - 1):
        source = list(connected_components[i])[0]
        target = list(connected_components[i + 1])[0]
        graph.add_edge(
            source, target,
            weight=0.1,
            color="gray",
            title="추가 연결"
        )

# Visualize with PyVis
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Customize physics for better layout
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.4,
      "springLength": 200,
      "springConstant": 0.1
    },
    "minVelocity": 0.5
  }
}
""")

# Save visualization
output_file = "chapter_by_code_flow_graph.html"
net.write_html(output_file)
print(f"Graph with chapter connections by 성취기준코드 flow saved to {output_file}")


Graph with chapter connections by 성취기준코드 flow saved to chapter_by_code_flow_graph.html


In [33]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = ["f_mchapter_id", "f_mchapter_nm", "성취기준코드"]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess 중단원명
data["f_mchapter_nm"] = (
    data["f_mchapter_nm"]
    .str.strip()  # 공백 제거
    .str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)  # 특수문자 제거
)

# Sort data by 성취기준코드 to establish flow
data = data.sort_values(by="성취기준코드").reset_index(drop=True)

# Map 성취기준코드 to 중단원 ID and Name
code_to_id = data.set_index("성취기준코드")["f_mchapter_id"].to_dict()
id_to_name = data.set_index("f_mchapter_id")["f_mchapter_nm"].to_dict()

# Generate edges based on 성취기준코드 flow
edges = []
previous_id = None
for _, row in data.iterrows():
    current_id = row["f_mchapter_id"]
    if previous_id and previous_id != current_id:
        edges.append((previous_id, current_id))
    previous_id = current_id

# Extract unique 중단원 IDs
unique_ids = data["f_mchapter_id"].drop_duplicates().tolist()

# Create the graph
graph = nx.DiGraph()

# Add nodes for 중단원 ID with labels as 중단원명
for node_id in unique_ids:
    graph.add_node(
        node_id,
        label=id_to_name[node_id],
        color="green",
        size=30
    )

# Add edges for 중단원 선후행 관계
for edge in edges:
    graph.add_edge(
        edge[0], edge[1],
        weight=1.0,
        color="blue",
        title="성취기준코드 흐름 기반 중단원 선후행 관계"
    )

# Add edges to ensure all nodes are connected if needed
connected_components = list(nx.weakly_connected_components(graph))
if len(connected_components) > 1:
    for i in range(len(connected_components) - 1):
        source = list(connected_components[i])[0]
        target = list(connected_components[i + 1])[0]
        graph.add_edge(
            source, target,
            weight=0.1,
            color="gray",
            title="추가 연결"
        )

# Visualize with PyVis
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Customize physics for better layout
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.4,
      "springLength": 200,
      "springConstant": 0.1
    },
    "minVelocity": 0.5
  }
}
""")

# Save visualization
output_file = "chapter_by_code_flow_graph_with_id.html"
net.write_html(output_file)
print(f"Graph with chapter connections by f_mchapter_id saved to {output_file}")


Graph with chapter connections by f_mchapter_id saved to chapter_by_code_flow_graph_with_id.html


### 위의 코드로 돌린 후, 중단원명이 다 표현되는지 확인하는 코드

In [53]:
# 그래프의 노드 이름 확인
graph_nodes = [node for node in graph.nodes]
print("그래프에 포함된 중단원 개수:", len(graph_nodes))
print("그래프에 포함된 중단원:", graph_nodes)

# 데이터에서 고유한 중단원 확인
unique_mchapters = data["f_mchapter_nm"].unique()
print("데이터에서 고유한 중단원 개수:", len(unique_mchapters))
print("데이터에서 고유한 중단원:", unique_mchapters)

# 누락된 중단원 확인
missing_chapters = set(unique_mchapters) - set(graph_nodes)
if missing_chapters:
    print("누락된 중단원:", missing_chapters)
else:
    print("모든 중단원이 그래프에 포함되어 있습니다.")


그래프에 포함된 중단원 개수: 94
그래프에 포함된 중단원: ['14201779', '14201780', '14201781', '14201782', '14201783', '14201784', '14201785', '14201786', '14201787', '14201788', '14201789', '14201790', '14201791', '14201792', '14201793', '14201794', '14201795', '14201796', '14201797', '14201798', '14201799', '14201800', '14201801', '14201802', '14201803', '14201804', '14201805', '14201806', '14201807', '14201808', '14201809', '14201810', '14201811', '14201812', '14201813', '14201814', '14201815', '14201816', '14201817', '14201818', '14201819', '14201820', '14201821', '14201857', '14201858', '14201859', '14201860', '14201861', '14201862', '14201863', '14201864', '14201865', '14201866', '14201867', '14201868', '14201869', '14201870', '14201871', '14201872', '14201873', '14201874', '14201875', '14201876', '14201877', '14201878', '14201879', '14201880', '14201881', '14201882', '14201883', '14201884', '14201885', '14201886', '14201887', '14201888', '14201889', '14201890', '14201891', '14201892', '14201893', '1420

### Sentence-BERT와 TF-IDF 입력 데이터 분리 + 유사도 가장 높은 거 하나만 연결

In [47]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준 흐름 기반 관계 생성
flow_edges = []
previous_id = None
for _, row in data.iterrows():
    current_id = row["f_mchapter_id"]
    if previous_id and previous_id != current_id:
        flow_edges.append((previous_id, current_id))
    previous_id = current_id

# 유사도 기반 관계 생성 (가장 높은 유사도 하나만 유지)
threshold = 0.95
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가
for chapter_id, chapter_name in zip(data["f_mchapter_id"], data["f_mchapter_nm"]):
    graph.add_node(
        str(chapter_id),  # ID는 문자열로 변환
        label=chapter_name,
        color="green",
        size=30
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        str(source), str(target),
        color="blue",
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true, 
    "stabilization": {
      "enabled": true, 
      "iterations": 1000, 
      "fit": true
    },
    "barnesHut": {
      "gravitationalConstant": -8000, 
      "centralGravity": 0.3, 
      "springLength": 500, 
      "springConstant": 0.05
    },
    "minVelocity": 0.1
  },
  "nodes": {
    "shape": "dot",
    "size": 20
  },
  "edges": {
    "smooth": {
      "type": "continuous"
    }
  }
}
''')

# Save visualization
output_file = "final_chapter_graph.html"
net.write_html(output_file)
print(f"Graph with integrated flow and similarity saved to {output_file}")


Graph with integrated flow and similarity saved to final_chapter_graph.html


### Sentence-BERT와 TF-IDF 입력 데이터 분리 + 유사도 가장 높은 거 하나만 연결 + 학년학기별 색상 부여

In [ ]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준 흐름 기반 관계 생성
flow_edges = []
previous_id = None
for _, row in data.iterrows():
    current_id = row["f_mchapter_id"]
    if previous_id and previous_id != current_id:
        flow_edges.append((previous_id, current_id))
    previous_id = current_id

# 유사도 기반 관계 생성 (가장 높은 유사도 하나만 유지)
threshold = 0.95
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 노드 색상 매핑
def get_node_color(subject_id):
    color_map = {
        2212: "red",
        2213: "blue",
        2214: "green",
        2215: "yellow"
    }
    return color_map.get(subject_id, "gray")

# 노드 추가
for chapter_id, chapter_name, subject_id in zip(data["f_mchapter_id"], data["f_mchapter_nm"], data["f_subject_id"]):
    graph.add_node(
        str(chapter_id),  # ID는 문자열로 변환
        label=chapter_name,
        color=get_node_color(subject_id),
        size=30
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        str(source), str(target),
        color="blue",
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true, 
    "stabilization": {
      "enabled": true, 
      "iterations": 1000, 
      "fit": true
    },
    "barnesHut": {
      "gravitationalConstant": -8000, 
      "centralGravity": 0.3, 
      "springLength": 500, 
      "springConstant": 0.05
    },
    "minVelocity": 0.1
  },
  "nodes": {
    "shape": "dot",
    "size": 20
  },
  "edges": {
    "smooth": {
      "type": "continuous"
    }
  }
}
''')

# Save visualization
output_file = "final_chapter_graph2.html"
net.write_html(output_file)
print(f"Graph with integrated flow and similarity saved to {output_file}")


Graph with integrated flow and similarity saved to final_chapter_graph2.html


### final_chapter_graph2 + 색상별 즉 학년 학기별 군집화 = final_chapter_graph3

In [76]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준 흐름 기반 관계 생성
flow_edges = []
previous_id = None
for _, row in data.iterrows():
    current_id = row["f_mchapter_id"]
    if previous_id and previous_id != current_id:
        flow_edges.append((previous_id, current_id))
    previous_id = current_id

# 유사도 기반 관계 생성
threshold = 0.95
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 색상 및 그룹 설정
def get_node_color(subject_id):
    color_map = {
        2212: "#FF4444",  # 더 선명한 빨강
        2213: "#4444FF",  # 더 선명한 파랑
        2214: "#44FF44",  # 더 선명한 초록
        2215: "#FFFF44"   # 더 선명한 노랑
    }
    return color_map.get(subject_id, "#GRAY")

# 노드를 subject_id별로 그룹화
grouped_data = data.groupby("f_subject_id")
y_positions = {sid: idx * 100 for idx, sid in enumerate(data["f_subject_id"].unique())}

# 노드 추가 (위치 정보 포함)
for chapter_id, chapter_name, subject_id in zip(data["f_mchapter_id"], data["f_mchapter_nm"], data["f_subject_id"]):
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color=get_node_color(subject_id),
        size=30,
        group=str(subject_id)  # 그룹 설정
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        str(source), str(target),
        color="#aaaaff",  # 연한 파란색으로 변경
        width=1,
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -100,
      "centralGravity": 0.01,
      "springLength": 200,
      "springConstant": 0.08,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  },
  "groups": {
    "2212": {"color": "#FF4444"},
    "2213": {"color": "#4444FF"},
    "2214": {"color": "#44FF44"},
    "2215": {"color": "#FFFF44"}
  }
}
''')

# Save visualization
output_file = "final_chapter_graph3.html"
net.write_html(output_file)
print(f"Graph with integrated flow and similarity saved to {output_file}")

Graph with integrated flow and similarity saved to final_chapter_graph3.html


### bert embedding으로 군집화

In [86]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
from sklearn.cluster import KMeans
import numpy as np
from sklearn.manifold import TSNE

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)

# BERT 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
embeddings_np = bert_embeddings.cpu().numpy()

# K-means 군집화
n_clusters = 4  # subject_id 개수와 동일하게 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(embeddings_np)

# t-SNE로 차원 축소 (전체 데이터에 대해 한 번만 실행)
tsne = TSNE(
    n_components=2,
    perplexity=min(30, len(embeddings_np) - 1),  # perplexity 값 조정
    random_state=42,
    n_iter=1000
)
embeddings_2d = tsne.fit_transform(embeddings_np)

# 성취기준 흐름 기반 관계 생성
flow_edges = []
previous_id = None
for _, row in data.iterrows():
    current_id = row["f_mchapter_id"]
    if previous_id and previous_id != current_id:
        flow_edges.append((previous_id, current_id))
    previous_id = current_id

# 유사도 기반 관계 생성
cosine_sim = cosine_similarity(embeddings_np)
threshold = 0.95
similarity_edges = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if cosine_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 군집별 색상 매핑
cluster_colors = ['#FF4444', '#4444FF', '#44FF44', '#FFFF44']

# 노드 추가 (임베딩 기반 위치 정보 포함)
for idx, (chapter_id, chapter_name) in enumerate(zip(data["f_mchapter_id"], data["f_mchapter_nm"])):
    # t-SNE 좌표를 기반으로 위치 설정
    x, y = embeddings_2d[idx]
    
    # 스케일 조정 (더 넓은 공간에 분포하도록)
    x = x * 10
    y = y * 10
    
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color=cluster_colors[cluster_labels[idx]],
        size=30,
        x=float(x),
        y=float(y),
        group=str(cluster_labels[idx])
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        str(source), str(target),
        color="#aaaaff",
        width=1,
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  },
  "groups": {
    "0": {"color": "#FF4444"},
    "1": {"color": "#4444FF"},
    "2": {"color": "#44FF44"},
    "3": {"color": "#FFFF44"}
  }
}
''')

# 군집화 결과 데이터프레임 생성
cluster_info = pd.DataFrame({
    'chapter_id': data['f_mchapter_id'],
    'chapter_name': data['f_mchapter_nm'],
    'subject_id': data['f_subject_id'],
    'cluster': cluster_labels
})

# 중복 제거 (chapter_name 기준으로 고유값만 유지)
cluster_info_unique = cluster_info.drop_duplicates(subset=['chapter_name'])

# 군집별 데이터 요약 출력
print("\n군집화 결과 분석:")
for cluster in range(n_clusters):
    cluster_data = cluster_info_unique[cluster_info_unique['cluster'] == cluster]
    print(f"\n클러스터 {cluster} ({len(cluster_data)} 개 노드):")
    print(cluster_data[['chapter_name', 'subject_id']].to_string(index=False))

# 저장할 경우 (옵션)
output_file = "clustered_results.csv"
cluster_info_unique.to_csv(output_file, index=False)
print(f"\n정리된 군집화 결과를 {output_file}에 저장했습니다.")


# Save visualization
output_file_html = "final_chapter_graph_clustered.html"
net.write_html(output_file_html)
print(f"\nGraph with embedding-based clustering saved to {output_file_html}")


c:\Users\USER2\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(



군집화 결과 분석:

클러스터 0 (11 개 노드):
chapter_name  subject_id
     두수의크기비교        2212
       덧셈과뺄셈        2212
    덧셈과뺄셈의관계        2214
       의값구하기        2214
       세수의계산        2214
   받아올림이있는덧셈        2213
   받아내림이있는뺄셈        2213
       덧셈하기2        2213
       덧셈하기3        2213
       뺄셈하기2        2213
       뺄셈하기3        2213

클러스터 1 (31 개 노드):
       chapter_name  subject_id
          12345알아보기        2212
           6789알아보기        2212
1만큼더큰수와1만큼더작은수0알아보기        2212
        덧셈과뺄셈이야기만들기        2212
                 덧셈        2212
                 뺄셈        2212
             10알아보기        2212
             십몇알아보기        2212
               묶어세기        2214
           곱셈식을활용하기        2214
             몇십알아보기        2213
         99까지의수알아보기        2213
           한자리세수의계산        2213
             두수를더하기        2213
           10이되는더하기        2213
             10에서빼기        2213
          10을만들어더하기        2213
          몇시30분알아보기        2213
         몇시몇시30분의응용        2213
          

### 원래 유사도 검사 + k-means 고정 & n_init 수 증가

In [91]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
from sklearn.cluster import KMeans
import numpy as np
from sklearn.manifold import TSNE

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준 흐름 기반 관계 생성
flow_edges = []
previous_id = None
for _, row in data.iterrows():
    current_id = row["f_mchapter_id"]
    if previous_id and previous_id != current_id:
        flow_edges.append((previous_id, current_id))
    previous_id = current_id

# 유사도 기반 관계 생성 (가장 높은 유사도 하나만 유지)
threshold = 0.95
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 군집별 색상 매핑
cluster_colors = ['#FF4444', '#4444FF', '#44FF44', '#FFFF44']

# K-means 군집화
n_clusters = 4  # subject_id 개수와 동일하게 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=20)
cluster_labels = kmeans.fit_predict(bert_embeddings.cpu().numpy())

# t-SNE로 차원 축소 (전체 데이터에 대해 한 번만 실행)
tsne = TSNE(
    n_components=2,
    perplexity=min(30, len(bert_embeddings) - 1),  # perplexity 값 조정
    random_state=42,
    n_iter=1000
)
embeddings_2d = tsne.fit_transform(bert_embeddings.cpu().numpy())

# 노드 추가 (임베딩 기반 위치 정보 포함)
for idx, (chapter_id, chapter_name) in enumerate(zip(data["f_mchapter_id"], data["f_mchapter_nm"])):
    # t-SNE 좌표를 기반으로 위치 설정
    x, y = embeddings_2d[idx]
    
    # 스케일 조정 (더 넓은 공간에 분포하도록)
    x = x * 10
    y = y * 10
    
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color=cluster_colors[cluster_labels[idx]],
        size=30,
        x=float(x),
        y=float(y),
        group=str(cluster_labels[idx])
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        str(source), str(target),
        color="#aaaaff",
        width=1,
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  },
  "groups": {
    "0": {"color": "#FF4444"},
    "1": {"color": "#4444FF"},
    "2": {"color": "#44FF44"},
    "3": {"color": "#FFFF44"}
  }
}
''')

# 군집화 결과 데이터프레임 생성
cluster_info = pd.DataFrame({
    'chapter_id': data['f_mchapter_id'],
    'chapter_name': data['f_mchapter_nm'],
    'subject_id': data['f_subject_id'],
    'cluster': cluster_labels
})

# 중복 제거 (chapter_name 기준으로 고유값만 유지)
cluster_info_unique = cluster_info.drop_duplicates(subset=['chapter_name'])

# 군집별 데이터 요약 출력
print("\n군집화 결과 분석:")
for cluster in range(n_clusters):
    cluster_data = cluster_info_unique[cluster_info_unique['cluster'] == cluster]
    print(f"\n클러스터 {cluster} ({len(cluster_data)} 개 노드):")
    print(cluster_data[['chapter_name', 'subject_id']].to_string(index=False))

# 저장할 경우 (옵션)
output_file = "clustered_results.csv"
cluster_info_unique.to_csv(output_file, index=False)
print(f"\n정리된 군집화 결과를 {output_file}에 저장했습니다.")

# Save visualization
output_file_html = "final_chapter_graph_clustered2.html"
net.write_html(output_file_html)
print(f"\nGraph with embedding-based clustering saved to {output_file_html}")


c:\Users\USER2\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(



군집화 결과 분석:

클러스터 0 (61 개 노드):
       chapter_name  subject_id
          12345알아보기        2212
           6789알아보기        2212
            9까지수의순서        2212
1만큼더큰수와1만큼더작은수0알아보기        2212
         여러가지모양찾아보기        2212
         여러가지모양알아보기        2212
        덧셈과뺄셈이야기만들기        2212
                 덧셈        2212
                 뺄셈        2212
             길이비교하기        2212
             무게비교하기        2212
             넓이비교하기        2212
         담을수있는양비교하기        2212
             높이비교하기        2212
              키비교하기        2212
             10알아보기        2212
             십몇알아보기        2212
          50까지의수의순서        2212
               뛰어세기        2214
                 도형        2214
               쌓기나무        2214
        여러가지단위길이로재기        2214
             자로길이재기        2214
       길이를어림해보고재어보기        2214
               분류하기        2214
               묶어세기        2214
            곱셈식알아보기        2214
           곱셈식을활용하기        2214
             몇십알아보기        2213
         

### 화살표 방향 : 시간의 흐름, 즉 학년 학기 순을 거스르지 않아야 함!

In [95]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
from sklearn.cluster import KMeans
import numpy as np
from sklearn.manifold import TSNE

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준 흐름 기반 관계 생성 (f_subject_id를 고려한 방향 설정)
flow_edges = []
for i in range(len(data) - 1):
    current_row = data.iloc[i]
    next_row = data.iloc[i + 1]
    
    # f_subject_id 값 기준으로 방향 설정
    if current_row["f_subject_id"] < next_row["f_subject_id"]:
        flow_edges.append((current_row["f_mchapter_id"], next_row["f_mchapter_id"]))

# 유사도 기반 관계 생성 (가장 높은 유사도 하나만 유지)
threshold = 0.95
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 군집별 색상 매핑
cluster_colors = ['#FF4444', '#4444FF', '#44FF44', '#FFFF44']

# K-means 군집화
n_clusters = 4  # subject_id 개수와 동일하게 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=20)
cluster_labels = kmeans.fit_predict(bert_embeddings.cpu().numpy())

# t-SNE로 차원 축소 (전체 데이터에 대해 한 번만 실행)
tsne = TSNE(
    n_components=2,
    perplexity=min(30, len(bert_embeddings) - 1),  # perplexity 값 조정
    random_state=42,
    n_iter=1000
)
embeddings_2d = tsne.fit_transform(bert_embeddings.cpu().numpy())

# 노드 추가 (임베딩 기반 위치 정보 포함)
for idx, (chapter_id, chapter_name) in enumerate(zip(data["f_mchapter_id"], data["f_mchapter_nm"])):
    # t-SNE 좌표를 기반으로 위치 설정
    x, y = embeddings_2d[idx]
    
    # 스케일 조정 (더 넓은 공간에 분포하도록)
    x = x * 10
    y = y * 10
    
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color=cluster_colors[cluster_labels[idx]],
        size=30,
        x=float(x),
        y=float(y),
        group=str(cluster_labels[idx])
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        str(source), str(target),
        color="#aaaaff",
        width=1,
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  },
  "groups": {
    "0": {"color": "#FF4444"},
    "1": {"color": "#4444FF"},
    "2": {"color": "#44FF44"},
    "3": {"color": "#FFFF44"}
  }
}
''')

# 군집화 결과 데이터프레임 생성
cluster_info = pd.DataFrame({
    'chapter_id': data['f_mchapter_id'],
    'chapter_name': data['f_mchapter_nm'],
    'subject_id': data['f_subject_id'],
    'cluster': cluster_labels
})

# 중복 제거 (chapter_name 기준으로 고유값만 유지)
cluster_info_unique = cluster_info.drop_duplicates(subset=['chapter_name'])

# 군집별 데이터 요약 출력
print("\n군집화 결과 분석:")
for cluster in range(n_clusters):
    cluster_data = cluster_info_unique[cluster_info_unique['cluster'] == cluster]
    print(f"\n클러스터 {cluster} ({len(cluster_data)} 개 노드):")
    print(cluster_data[['chapter_name', 'subject_id']].to_string(index=False))

# 저장할 경우 (옵션)
output_file = "clustered_results3.csv"
cluster_info_unique.to_csv(output_file, index=False)
print(f"\n정리된 군집화 결과를 {output_file}에 저장했습니다.")

# Save visualization
output_file_html = "final_chapter_graph_clustered3.html"
net.write_html(output_file_html)
print(f"\nGraph with embedding-based clustering saved to {output_file_html}")


c:\Users\USER2\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(



군집화 결과 분석:

클러스터 0 (61 개 노드):
       chapter_name  subject_id
          12345알아보기        2212
           6789알아보기        2212
            9까지수의순서        2212
1만큼더큰수와1만큼더작은수0알아보기        2212
         여러가지모양찾아보기        2212
         여러가지모양알아보기        2212
        덧셈과뺄셈이야기만들기        2212
                 덧셈        2212
                 뺄셈        2212
             길이비교하기        2212
             무게비교하기        2212
             넓이비교하기        2212
         담을수있는양비교하기        2212
             높이비교하기        2212
              키비교하기        2212
             10알아보기        2212
             십몇알아보기        2212
          50까지의수의순서        2212
               뛰어세기        2214
                 도형        2214
               쌓기나무        2214
        여러가지단위길이로재기        2214
             자로길이재기        2214
       길이를어림해보고재어보기        2214
               분류하기        2214
               묶어세기        2214
            곱셈식알아보기        2214
           곱셈식을활용하기        2214
             몇십알아보기        2213
         

### 화살표 방향(역순흐름X) & 2022년 개정 수학 교육부 성취기준 4개 영역(수와연산,변화와관계,도형과측정,자료와가능성)

In [62]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
import json

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 군집화 및 학년 학기 색상 매핑
area_colors = {
    "수와 연산": "#FFCCCC",
    "변화와 관계": "#CCCCFF",
    "도형과 측정": "#CCFFCC",
    "자료와 가능성": "#FFFFCC",
    "기타": "#CCCCCC"
}
subject_colors = {
    2212: "#FF0000",  # 1학년 1학기
    2213: "#0000FF",  # 1학년 2학기
    2214: "#00FF00",  # 2학년 1학기
    2215: "#FFFF00"   # 2학년 2학기
}

def get_area_by_code(code):
    if code.startswith("2수01"):
        return "수와 연산"
    elif code.startswith("2수02"):
        return "변화와 관계"
    elif code.startswith("2수03"):
        return "도형과 측정"
    elif code.startswith("2수04"):
        return "자료와 가능성"
    else:
        return "기타"

data["area"] = data["성취기준코드"].apply(get_area_by_code)
data["color"] = data["area"].map(area_colors)
data["border_color"] = data["f_subject_id"].map(subject_colors)

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 흐름 기반 엣지 생성
flow_edges = []
for i in range(len(data) - 1):
    current_row = data.iloc[i]
    next_row = data.iloc[i + 1]
    if (current_row["f_subject_id"] < next_row["f_subject_id"] or
        (current_row["f_subject_id"] == next_row["f_subject_id"] and
         current_row["성취기준코드"] < next_row["성취기준코드"])):
        flow_edges.append((current_row["f_mchapter_id"], next_row["f_mchapter_id"]))

# 유사도 기반 관계 생성
threshold = 0.85
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가
nodes_json = []
for idx, (chapter_id, chapter_name, color, border_color) in enumerate(zip(
    data["f_mchapter_id"], data["f_mchapter_nm"], data["color"], data["border_color"]
)):
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color={
            "background": color,
            "border": border_color
        },
        size=30,
        borderWidth=2
    )
    nodes_json.append({
        "id": chapter_id,
        "name": chapter_name,
        "area": data.loc[idx, "area"],
        "subject_id": data.loc[idx, "f_subject_id"],
        "color": color,
        "border_color": border_color
    })

# 엣지 추가
edges_json = []
for source, target in final_edges:
    if str(source) in graph.nodes and str(target) in graph.nodes and source != target:
        graph.add_edge(
            str(source), str(target),
            color="#aaaaff",
            width=1,
            title="선후행 관계 및 유사도 기반 연결"
        )
        edges_json.append({
            "source": source,
            "target": target
        })

# PyVis 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  }
}
''')

# 그래프 저장
output_file_html = "final_chapter_graph_clustered4.html"
net.write_html(output_file_html)
print(f"\nGraph with embedding-based clustering saved to {output_file_html}")




Graph with embedding-based clustering saved to final_chapter_graph_clustered4.html


In [107]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^\uac00-\ud7a3a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 흐름 기반 엣지 생성
flow_edges = []
for i in range(len(data) - 1):
    current_row = data.iloc[i]
    next_row = data.iloc[i + 1]
    if (current_row["f_subject_id"] < next_row["f_subject_id"] or
        (current_row["f_subject_id"] == next_row["f_subject_id"] and
         current_row["성취기준코드"] < next_row["성취기준코드"])):
        flow_edges.append((current_row["f_mchapter_id"], next_row["f_mchapter_id"]))

# 유사도 기반 관계 생성
threshold = 0.85
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source_id = str(data.iloc[i]["f_mchapter_id"])
            target_id = str(data.iloc[j]["f_mchapter_id"])
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# Combine edges
final_edges = list(set(flow_edges + similarity_edges))

# 노드 JSON 생성
nodes_json = []
for idx, (chapter_id, chapter_name, subject_id, 성취기준코드) in enumerate(zip(
    data["f_mchapter_id"], data["f_mchapter_nm"], data["f_subject_id"], data["성취기준코드"]
)):
    nodes_json.append({
        "id": str(chapter_id),
        "name": chapter_name,
        "subject_id": int(subject_id),
        "code": 성취기준코드
    })

# 엣지 JSON 생성
edges_json = []
for source, target in final_edges:
    edges_json.append({
        "source": str(source),
        "target": str(target)
    })

# 결과 JSON 저장
output_json = {
    "nodes": nodes_json,
    "edges": edges_json
}
output_file = "graph_data.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_json, f, ensure_ascii=False, indent=4)

print(f"JSON data saved to {output_file}")


JSON data saved to graph_data.json


In [63]:
import numpy as np
print("유사도 매트릭스 샘플:\n", np.round(final_sim[:5, :5], 2))  # 상위 5개 행/열만 출력


유사도 매트릭스 샘플:
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [ ]:
threshold = 0.85  # 기준
similar_pairs = np.argwhere(final_sim >= threshold)
for i, j in similar_pairs:
    if i != j:
        print(f"Node {i} ↔ Node {j} : {final_sim[i, j]:.2f}")


In [65]:
print(data["area"].value_counts())


area
수와 연산      503
도형과 측정     237
자료와 가능성     32
변화와 관계      32
Name: count, dtype: int64


In [68]:
print(data["f_subject_id"].value_counts())


f_subject_id
2214    228
2213    197
2212    192
2215    187
Name: count, dtype: int64


In [67]:
print("Flow Edges Sample:\n", flow_edges[:5])  # 흐름 기반 엣지 상위 5개
print("Similarity Edges Sample:\n", similarity_edges[:5])  # 유사도 기반 엣지 상위 5개


Flow Edges Sample:
 [(14201780, 14201781), (14201781, 14201782), (14201782, 14201783), (14201783, 14201784), (14201784, 14201785)]
Similarity Edges Sample:
 [(14201779, 14201780), (14201780, 14201798), (14201781, 14201801), (14201782, 14201788), (14201783, 14201790)]


In [69]:
print(f"Total Edges: {len(final_edges)}")


Total Edges: 136


In [72]:
print(f"Flow edges 개수: {len(flow_edges)}")
print(f"Similarity edges 개수: {len(similarity_edges)}")



Flow edges 개수: 68
Similarity edges 개수: 77


In [73]:
final_edges = list(set(flow_edges + similarity_edges))
print(f"최종 엣지 개수: {len(final_edges)}")


최종 엣지 개수: 136


In [76]:
unique_chapters = data["f_mchapter_id"].nunique()
print(f"고유 중단원 노드 수: {unique_chapters}")



고유 중단원 노드 수: 94


In [77]:
duplicated_rows = data.duplicated(subset=["f_mchapter_id"], keep=False).sum()
print(f"중복된 데이터 수: {duplicated_rows}")


중복된 데이터 수: 800


In [78]:
print(f"전체 데이터 행 수: {len(data)}")
print(f"고유 중단원 노드 수: {unique_chapters}")


전체 데이터 행 수: 804
고유 중단원 노드 수: 94


In [79]:
print(f"Flow edges 개수: {len(flow_edges)}")
print(f"Similarity edges 개수: {len(similarity_edges)}")
print(f"최종 엣지 개수: {len(final_edges)}")


Flow edges 개수: 68
Similarity edges 개수: 77
최종 엣지 개수: 136


In [80]:
isolated_nodes = [
    i for i in range(len(data))
    if all(final_sim[i, j] < threshold for j in range(len(data)) if i != j)
]
print(f"유사도 조건 미달 노드 수: {len(isolated_nodes)}")


유사도 조건 미달 노드 수: 0


### 엣지 연결 93개

In [85]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
import json

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^\uac00-\ud7a3a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 군집화 및 학년 학기 색상 매핑
area_colors = {
    "수와 연산": "#FFCCCC",
    "변화와 관계": "#CCCCFF",
    "도형과 측정": "#CCFFCC",
    "자료와 가능성": "#FFFFCC",
    "기타": "#CCCCCC"
}
subject_colors = {
    2212: "#FF0000",  # 1학년 1학기
    2213: "#0000FF",  # 1학년 2학기
    2214: "#00FF00",  # 2학년 1학기
    2215: "#FFFF00"   # 2학년 2학기
}

def get_area_by_code(code):
    if code.startswith("2수01"):
        return "수와 연산"
    elif code.startswith("2수02"):
        return "변화와 관계"
    elif code.startswith("2수03"):
        return "도형과 측정"
    elif code.startswith("2수04"):
        return "자료와 가능성"
    else:
        return "기타"

data["area"] = data["성취기준코드"].apply(get_area_by_code)
data["color"] = data["area"].map(area_colors)
data["border_color"] = data["f_subject_id"].map(subject_colors)

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 노드를 시간 흐름 순서로 정렬 후 선형 연결
sorted_nodes = data.sort_values(by=["f_subject_id", "성취기준코드"])["f_mchapter_id"].unique()
sequential_edges = [(sorted_nodes[i], sorted_nodes[i + 1]) for i in range(len(sorted_nodes) - 1)]

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가
for idx, (chapter_id, chapter_name, color, border_color) in enumerate(zip(
    data["f_mchapter_id"], data["f_mchapter_nm"], data["color"], data["border_color"]
)):
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color={
            "background": color,
            "border": border_color
        },
        size=30,
        borderWidth=2
    )

# 엣지 추가
for source, target in sequential_edges:
    graph.add_edge(
        str(source), str(target),
        color="#aaaaff",
        width=1,
        title="뭐지지"
    )

# PyVis 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''
{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  }
}
''')

# 그래프 저장
output_file_html = "linear_graph.html"
net.write_html(output_file_html)
print(f"\nGraph with sequential connections saved to {output_file_html}")



Graph with sequential connections saved to linear_graph.html


In [88]:
# 순차적 연결(시간 흐름 기반) 엣지 개수 확인
print(f"Sequential edges 개수: {len(sequential_edges)}")  # 시간 흐름 기반 선형 연결의 개수


Sequential edges 개수: 93


In [90]:
sorted_nodes = data.sort_values(by=["f_subject_id", "성취기준코드"])["f_mchapter_id"].unique()


In [92]:
source_row = data[data["f_mchapter_id"] == 14201779]
target_row = data[data["f_mchapter_id"] == 14201780]

print("Source 노드 정보:", source_row[["f_subject_id", "성취기준코드"]])
print("Target 노드 정보:", target_row[["f_subject_id", "성취기준코드"]])


Source 노드 정보:    f_subject_id   성취기준코드
0          2212  2수01-01
1          2212  2수01-01
2          2212  2수01-01
3          2212  2수01-01
4          2212  2수01-01
5          2212  2수01-01
Target 노드 정보:     f_subject_id   성취기준코드
6           2212  2수01-01
7           2212  2수01-01
8           2212  2수01-01
9           2212  2수01-01
10          2212  2수01-01


In [93]:
data = data.sort_values(by=["f_subject_id", "성취기준코드"])
sorted_nodes = data["f_mchapter_id"].unique()


In [94]:
print(data[["f_mchapter_id", "f_subject_id", "성취기준코드"]].head(10))


   f_mchapter_id  f_subject_id   성취기준코드
0       14201779          2212  2수01-01
1       14201779          2212  2수01-01
2       14201779          2212  2수01-01
3       14201779          2212  2수01-01
4       14201779          2212  2수01-01
5       14201779          2212  2수01-01
6       14201780          2212  2수01-01
7       14201780          2212  2수01-01
8       14201780          2212  2수01-01
9       14201780          2212  2수01-01


In [96]:
for source, target in sequential_edges:
    source_row = data[data["f_mchapter_id"] == source]
    target_row = data[data["f_mchapter_id"] == target]

    # 학기 순서만 비교
    assert source_row["f_subject_id"].values[0] <= target_row["f_subject_id"].values[0], \
        f"학기 순서 오류: {source} → {target}"
    
    # 동일 학기 내에서 성취기준코드 비교
    if source_row["f_subject_id"].values[0] == target_row["f_subject_id"].values[0]:
        if source_row["성취기준코드"].values[0] >= target_row["성취기준코드"].values[0]:
            print(f"코드 순서 경고(무시 가능): {source} → {target}")


코드 순서 경고(무시 가능): 14201779 → 14201780
코드 순서 경고(무시 가능): 14201780 → 14201798
코드 순서 경고(무시 가능): 14201798 → 14201799
코드 순서 경고(무시 가능): 14201781 → 14201801
코드 순서 경고(무시 가능): 14201801 → 14201802
코드 순서 경고(무시 가능): 14201786 → 14201787
코드 순서 경고(무시 가능): 14201782 → 14201788
코드 순서 경고(무시 가능): 14201788 → 14201789
코드 순서 경고(무시 가능): 14201789 → 14201790
코드 순서 경고(무시 가능): 14201792 → 14201793
코드 순서 경고(무시 가능): 14201793 → 14201794
코드 순서 경고(무시 가능): 14201794 → 14201795
코드 순서 경고(무시 가능): 14201795 → 14201796
코드 순서 경고(무시 가능): 14201796 → 14201797
코드 순서 경고(무시 가능): 14201857 → 14201858
코드 순서 경고(무시 가능): 14201859 → 14201860
코드 순서 경고(무시 가능): 14201860 → 14201861
코드 순서 경고(무시 가능): 14201862 → 14201874
코드 순서 경고(무시 가능): 14201863 → 14201864
코드 순서 경고(무시 가능): 14201864 → 14201865
코드 순서 경고(무시 가능): 14201865 → 14201866
코드 순서 경고(무시 가능): 14201866 → 14201867
코드 순서 경고(무시 가능): 14201867 → 14201881
코드 순서 경고(무시 가능): 14201881 → 14201884
코드 순서 경고(무시 가능): 14201884 → 14201885
코드 순서 경고(무시 가능): 14201885 → 14201888
코드 순서 경고(무시 가능): 14201875 → 14201876
코

In [98]:
print(f"노드 개수: {len(data['f_mchapter_id'].unique())}")
print(f"엣지 개수: {len(sequential_edges)}")
assert len(sequential_edges) == len(data['f_mchapter_id'].unique()) - 1, "엣지 개수가 예상과 다릅니다!"


노드 개수: 94
엣지 개수: 93


In [102]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
import json

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^\uac00-\ud7a3a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 군집화 및 학년 학기 색상 매핑
area_colors = {
    "수와 연산": "#FFCCCC",
    "변화와 관계": "#CCCCFF",
    "도형과 측정": "#CCFFCC",
    "자료와 가능성": "#FFFFCC",
    "기타": "#CCCCCC"
}
subject_colors = {
    2212: "#FF0000",  # 1학년 1학기
    2213: "#0000FF",  # 1학년 2학기
    2214: "#00FF00",  # 2학년 1학기
    2215: "#FFFF00"   # 2학년 2학기
}

def get_area_by_code(code):
    if code.startswith("2수01"):
        return "수와 연산"
    elif code.startswith("2수02"):
        return "변화와 관계"
    elif code.startswith("2수03"):
        return "도형과 측정"
    elif code.startswith("2수04"):
        return "자료와 가능성"
    else:
        return "기타"

data["area"] = data["성취기준코드"].apply(get_area_by_code)
data["color"] = data["area"].map(area_colors)
data["border_color"] = data["f_subject_id"].map(subject_colors)

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 노드를 시간 흐름 순서로 정렬 후 선형 연결
sorted_nodes = data.sort_values(by=["f_subject_id", "성취기준코드"])["f_mchapter_id"].unique()
sequential_edges = [(sorted_nodes[i], sorted_nodes[i + 1]) for i in range(len(sorted_nodes) - 1)]

# 유사도 기반 연결 생성
threshold = 0.85  # 유사도 임계값
similarity_edges = []
for i in range(len(data)):
    for j in range(len(data)):
        if i != j and final_sim[i, j] >= threshold:  # 자기 자신 제외
            similarity_edges.append((data.iloc[i]["f_mchapter_id"], data.iloc[j]["f_mchapter_id"], final_sim[i, j]))

# 가장 강력한 연결 필터링
strongest_edges = []
node_connections = {}

for source, target, sim_score in similarity_edges:
    if source not in node_connections or node_connections[source][1] < sim_score:
        node_connections[source] = (target, sim_score)

for source, (target, sim_score) in node_connections.items():
    if source != target:  # 자기 자신 제외
        strongest_edges.append((source, target))

# 시간 흐름과 유사도 기반 연결 통합
final_edges = list(set(sequential_edges + strongest_edges))

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가
for idx, (chapter_id, chapter_name, color, border_color) in enumerate(zip(
    data["f_mchapter_id"], data["f_mchapter_nm"], data["color"], data["border_color"]
)):
    graph.add_node(
        str(chapter_id),
        label=chapter_name,
        color={
            "background": color,
            "border": border_color
        },
        size=30,
        borderWidth=2
    )

# 엣지 추가
for source, target in final_edges:
    if source != target:  # 자기 자신을 가리키는 엣지 제거
        graph.add_edge(
            str(source), str(target),
            color="#ffa500",  # 점수 기반 최적 연결
            width=1,
            title="최적 연결"
        )

# PyVis 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# 물리적 레이아웃 설정
net.set_options('''{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  }
}''')

# 그래프 저장
output_file_html = "optimal_graph2.html"
net.write_html(output_file_html)
print(f"\nGraph with optimal connections saved to {output_file_html}")



Graph with optimal connections saved to optimal_graph2.html


In [104]:
# 첫 번째 코드
print(f"Flow edges: {len(flow_edges)}")
print(f"Similarity edges (First Code): {len(similarity_edges)}")
print(f"Final edges (First Code): {len(final_edges)}")

# 두 번째 코드
print(f"Sequential edges: {len(sequential_edges)}")
print(f"Similarity edges (Second Code): {len(strongest_edges)}")
print(f"Final edges (Second Code): {len(final_edges)}")



Flow edges: 68
Similarity edges (First Code): 38470
Final edges (First Code): 163
Sequential edges: 93
Similarity edges (Second Code): 72
Final edges (Second Code): 163


### 시간 제한 적용과 하지 않은 건 무슨 차이가 있나.... 

In [105]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^\uac00-\ud7a3a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 군집화 및 학년 학기 색상 매핑
area_colors = {
    "수와 연산": "#FFCCCC",
    "변화와 관계": "#CCCCFF",
    "도형과 측정": "#CCFFCC",
    "자료와 가능성": "#FFFFCC",
    "기타": "#CCCCCC"
}
subject_colors = {
    2212: "#FF0000",  # 1학년 1학기
    2213: "#0000FF",  # 1학년 2학기
    2214: "#00FF00",  # 2학년 1학기
    2215: "#FFFF00"   # 2학년 2학기
}

def get_area_by_code(code):
    if code.startswith("2수01"):
        return "수와 연산"
    elif code.startswith("2수02"):
        return "변화와 관계"
    elif code.startswith("2수03"):
        return "도형과 측정"
    elif code.startswith("2수04"):
        return "자료와 가능성"
    else:
        return "기타"

data["area"] = data["성취기준코드"].apply(get_area_by_code)
data["color"] = data["area"].map(area_colors)
data["border_color"] = data["f_subject_id"].map(subject_colors)

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 흐름 기반 엣지 생성
flow_edges = []
for i in range(len(data) - 1):
    current_row = data.iloc[i]
    next_row = data.iloc[i + 1]
    if (current_row["f_subject_id"] < next_row["f_subject_id"] or
        (current_row["f_subject_id"] == next_row["f_subject_id"] and
         current_row["성취기준코드"] < next_row["성취기준코드"])):
        flow_edges.append((current_row["f_mchapter_id"], next_row["f_mchapter_id"]))

# 유사도 기반 연결 생성 (시간 제한 적용 또는 미적용)
def create_similarity_edges(data, final_sim, threshold=0.85, time_restricted=False):
    similarity_edges = []
    for i in range(len(data)):
        for j in range(len(data)):
            if i != j and final_sim[i, j] >= threshold:
                source = data.iloc[i]
                target = data.iloc[j]
                # 시간 제한 조건
                if time_restricted:
                    if source["f_subject_id"] >= target["f_subject_id"]:  # 선행 학습 제한
                        continue
                similarity_edges.append((source["f_mchapter_id"], target["f_mchapter_id"]))
    return similarity_edges

# 유사도 기반 연결 생성
similarity_edges_no_time = create_similarity_edges(data, final_sim, time_restricted=False)
similarity_edges_with_time = create_similarity_edges(data, final_sim, time_restricted=True)

# 최적의 엣지 생성
final_edges_no_time = list(set(flow_edges + similarity_edges_no_time))
final_edges_with_time = list(set(flow_edges + similarity_edges_with_time))

# 그래프 생성 (시간 제한 미적용)
graph_no_time = nx.DiGraph()
for source, target in final_edges_no_time:
    if source != target:
        graph_no_time.add_edge(
            str(source), str(target),
            color="#ffa500",  # 점수 기반 최적 연결
            width=1,
            title="유사도 기반 연결 (시간 제한 없음)"
        )

# 그래프 생성 (시간 제한 적용)
graph_with_time = nx.DiGraph()
for source, target in final_edges_with_time:
    if source != target:
        graph_with_time.add_edge(
            str(source), str(target),
            color="#aaaaff",  # 점수 기반 최적 연결
            width=1,
            title="유사도 기반 연결 (시간 제한 적용)"
        )

# PyVis 시각화
net_no_time = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net_no_time.from_nx(graph_no_time)
net_no_time.set_options('''{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  }
}''')

net_with_time = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net_with_time.from_nx(graph_with_time)
net_with_time.set_options('''{
  "physics": {
    "enabled": true,
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 100,
      "springConstant": 0.04,
      "damping": 0.4,
      "avoidOverlap": 1
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "enabled": true,
      "iterations": 2000,
      "updateInterval": 25
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 25,
    "font": {
      "size": 14
    }
  },
  "edges": {
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    }
  }
}''')

# 그래프 저장
net_no_time.write_html("optimal_graph_no_time.html")
net_with_time.write_html("optimal_graph_with_time.html")
print(f"\nGraphs saved: 'optimal_graph_no_time.html' (No Time Restriction), 'optimal_graph_with_time.html' (With Time Restriction)")



Graphs saved: 'optimal_graph_no_time.html' (No Time Restriction), 'optimal_graph_with_time.html' (With Time Restriction)


### 결과 json으로 저장후 다시 나타내기

In [60]:
# Before saving to JSON, convert the data to standard Python types
nodes_json = [{
    "id": int(node["id"]),  # Convert int64 to regular Python int
    "name": str(node["name"]),
    "area": str(node["area"]),
    "subject_id": int(node["subject_id"]),
    "color": str(node["color"]),
    "border_color": str(node["border_color"])
} for node in nodes_json]

edges_json = [{
    "source": int(edge["source"]),  # Convert int64 to regular Python int
    "target": int(edge["target"])
} for edge in edges_json]

graph_data = {
    "nodes": nodes_json,
    "edges": edges_json
}

# Now save to JSON
with open('chapter_graph_data.json', 'w', encoding='utf-8') as f:
    json.dump(graph_data, f, ensure_ascii=False, indent=2)


In [11]:
target_chapter_id = data[data["f_mchapter_nm"] == "쌓기나무"]["f_mchapter_id"].values[0]
print("Flow Edges:", [(source, target) for source, target in flow_edges if source == target_chapter_id or target == target_chapter_id])
print("Similarity Edges:", [(source, target) for source, target in similarity_edges if source == target_chapter_id or target == target_chapter_id])


Flow Edges: []
Similarity Edges: []


In [ ]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Neo4j 연결 설정
neo4j_url = "bolt://localhost:7687"  # 네오4j 기본 URL
neo4j_user = "neo4j"  # 사용자 이름
neo4j_password = "12345678"  # 비밀번호 설정
graph = Graph(neo4j_url, auth=(neo4j_user, neo4j_password))

# Neo4j 데이터 초기화 (옵션: 기존 데이터 유지 시 생략 가능)
def clear_database():
    graph.run("MATCH (n) DETACH DELETE n")
    print("Neo4j 데이터베이스 초기화 완료.")

clear_database()  # 데이터 초기화

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess data
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^\uAC00-\uD7A3a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 학년 학기별 색상 매핑
subject_colors = {
    2212: "#FF9999",  # 1학년 1학기
    2213: "#9999FF",  # 1학년 2학기
    2214: "#99FF99",  # 2학년 1학기
    2215: "#FFFF99"   # 2학년 2학기
}
data["color"] = data["f_subject_id"].map(subject_colors)

# Sentence-BERT similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Combine similarities
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 흐름 기반 엣지 생성 (학년, 학기 순서와 성취기준 순서 기반)
flow_edges = []
for i in range(len(data) - 1):
    current_row = data.iloc[i]
    next_row = data.iloc[i + 1]
    if (current_row["f_subject_id"] < next_row["f_subject_id"] or
        (current_row["f_subject_id"] == next_row["f_subject_id"] and
         current_row["성취기준코드"] < next_row["성취기준코드"])):
        flow_edges.append((current_row["f_mchapter_id"], next_row["f_mchapter_id"]))

# 유사도 기반 관계 생성 (가장 높은 유사도 하나만 유지)
similarity_edges = []
node_max_connections = {}
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= 0.95:
            source_id = data.iloc[i]["f_mchapter_id"]
            target_id = data.iloc[j]["f_mchapter_id"]
            if source_id != target_id:
                if source_id not in node_max_connections or node_max_connections[source_id][1] < final_sim[i, j]:
                    node_max_connections[source_id] = (target_id, final_sim[i, j])

for source_id, (target_id, weight) in node_max_connections.items():
    similarity_edges.append((source_id, target_id))

# 모든 관계 통합
all_edges = list(set(flow_edges + similarity_edges))

# Neo4j 업로드 함수
def add_nodes_and_relationships(data, all_edges):
    for _, row in data.iterrows():
        node = Node(
            "Chapter",
            id=int(row["f_mchapter_id"]),
            name=row["f_mchapter_nm"],
            subject_id=int(row["f_subject_id"]),
            color=row["color"]
        )
        graph.merge(node, "Chapter", "id")

    for source, target in all_edges:
        if source != target:
            source_node = graph.nodes.match("Chapter", id=int(source)).first()
            target_node = graph.nodes.match("Chapter", id=int(target)).first()
            if source_node and target_node:
                relationship = Relationship(source_node, "PRECEDES", target_node)
                graph.merge(relationship)

# Neo4j에 데이터 추가
add_nodes_and_relationships(data, all_edges)
print("Neo4j 데이터 업로드 완료.")



Graph with embedding-based clustering saved to final_chapter_graph_clustered4.html


### neo4j 연결

In [124]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Neo4j 연결 설정
neo4j_url = "bolt://localhost:7687"  # 네오4j 기본 URL
neo4j_user = "neo4j"  # 사용자 이름
neo4j_password = "12345678"  # 비밀번호 설정
graph = Graph(neo4j_url, auth=(neo4j_user, neo4j_password))

# Neo4j 데이터 초기화 (기존 데이터 삭제)
def clear_database():
    graph.run("MATCH (n) DETACH DELETE n")
    print("Neo4j 데이터베이스 초기화 완료.")

clear_database()

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"
data = pd.read_csv(file_path)

# 데이터 컬럼 확인
required_columns = [
    "f_mchapter_id", "f_mchapter_nm", "f_subject_id", "성취기준코드", "성취기준 내용",
    "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# 데이터 전처리
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 학년 학기별 색상 매핑
subject_colors = {
    2212: "#FF9999",  # 1학년 1학기
    2213: "#9999FF",  # 1학년 2학기
    2214: "#99FF99",  # 2학년 1학기
    2215: "#FFFF99"   # 2학년 2학기
}
data["color"] = data["f_subject_id"].map(subject_colors)

# Sentence-BERT 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF 유사도 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# BERT와 TF-IDF 유사도 결합
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 선후행 관계 생성
flow_edges = []
for i in range(len(data) - 1):
    current_row = data.iloc[i]
    next_row = data.iloc[i + 1]
    if (current_row["f_subject_id"] < next_row["f_subject_id"] or
        (current_row["f_subject_id"] == next_row["f_subject_id"] and
         current_row["성취기준코드"] < next_row["성취기준코드"])):
        flow_edges.append((current_row["f_mchapter_id"], next_row["f_mchapter_id"]))

# 유사도 관계 생성 (가장 높은 유사도 하나만 유지)
similarity_edges = []
for i in range(len(data)):
    max_similarity = 0
    best_match = None
    for j in range(len(data)):
        if i != j and final_sim[i, j] > max_similarity:
            max_similarity = final_sim[i, j]
            best_match = data.iloc[j]["f_mchapter_id"]
    if best_match:
        similarity_edges.append((data.iloc[i]["f_mchapter_id"], best_match))

# Neo4j 노드 및 관계 추가
def add_nodes_and_relationships(data, flow_edges, similarity_edges):
    for _, row in data.iterrows():
        node = Node(
            "Chapter",
            id=int(row["f_mchapter_id"]),
            name=row["f_mchapter_nm"],
            subject_id=int(row["f_subject_id"]),
            color=row["color"]
        )
        graph.merge(node, "Chapter", "id")

    for source, target in flow_edges:
        if source != target:
            source_node = graph.nodes.match("Chapter", id=int(source)).first()
            target_node = graph.nodes.match("Chapter", id=int(target)).first()
            if source_node and target_node:
                relationship = Relationship(source_node, "PRECEDES", target_node)
                graph.merge(relationship)

    for source, target in similarity_edges:
        if source != target:
            source_node = graph.nodes.match("Chapter", id=int(source)).first()
            target_node = graph.nodes.match("Chapter", id=int(target)).first()
            if source_node and target_node:
                relationship = Relationship(source_node, "SIMILAR_TO", target_node)
                graph.merge(relationship)

# 데이터 추가
add_nodes_and_relationships(data, flow_edges, similarity_edges)
print("Neo4j 데이터 업로드 완료.")


Neo4j 데이터베이스 초기화 완료.
Neo4j 데이터 업로드 완료.


### final_chapter_graph2.html => nodes&edges

In [121]:
import pandas as pd

# 노드 데이터 저장
node_data = pd.DataFrame({
    "Node ID": [node for node in graph.nodes()],
    "Node Label": [graph.nodes[node]["label"] for node in graph.nodes()],
    "Node Color": [graph.nodes[node]["color"] for node in graph.nodes()]
})
node_output_file = "nodes_data.csv"
node_data.to_csv(node_output_file, index=False, encoding="utf-8-sig")
print(f"Node data saved to {node_output_file}")

# 엣지 데이터 저장
edge_data = pd.DataFrame({
    "Source": [edge[0] for edge in graph.edges()],
    "Target": [edge[1] for edge in graph.edges()],
    "Edge Color": [graph.edges[edge]["color"] for edge in graph.edges()],
    "Edge Title": [graph.edges[edge]["title"] for edge in graph.edges()]
})
edge_output_file = "edges_data.csv"
edge_data.to_csv(edge_output_file, index=False, encoding="utf-8-sig")
print(f"Edge data saved to {edge_output_file}")


TypeError: 'NodeMatcher' object is not callable

In [55]:
# 그래프에 포함된 노드 확인
graph_nodes = set(graph.nodes)

# 데이터에서 중단원명(f_mchapter_nm) 기준으로 고유 값 가져오기
unique_chapters = set(data["f_mchapter_id"].astype(str))

# 그래프에 표시되지 않은 중단원 찾기
missing_chapters = unique_chapters - graph_nodes

if len(missing_chapters) == 0:
    print("모든 중단원이 그래프에 표시되었습니다!")
else:
    print(f"그래프에 표시되지 않은 중단원 ID: {missing_chapters}")

# 그래프에 추가된 노드 개수와 데이터의 고유 중단원 수 비교
print(f"그래프 노드 개수: {len(graph_nodes)}")
print(f"데이터의 고유 중단원 개수: {len(unique_chapters)}")


모든 중단원이 그래프에 표시되었습니다!
그래프 노드 개수: 94
데이터의 고유 중단원 개수: 94


### Sentence-BERT와 TF-IDF 입력 데이터 분리:

- bert_text: 성취기준 내용, 성취수준 A, 성취수준 B, 성취수준 C만 사용.
- tfidf_text: 핵심키워드_v2만 사용.

In [55]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = ["f_mchapter_nm", "성취기준코드", "성취기준 내용", "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess 중단원명
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\s]", "", regex=True)

# 텍스트 결합
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF 유사도 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 두 가지 유사도 가중합
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준코드 흐름 기반 관계 생성
edges_from_code = []
previous_chapter = None
for _, row in data.iterrows():
    current_chapter = row["f_mchapter_nm"]
    if previous_chapter and previous_chapter != current_chapter:
        edges_from_code.append((previous_chapter, current_chapter))
    previous_chapter = current_chapter

# 유사도 기반 관계 생성
threshold = 0.8
edges_from_similarity = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):  # 한 방향만 설정
        if final_sim[i, j] >= threshold:
            source_chapter = data.iloc[i]["f_mchapter_nm"]
            target_chapter = data.iloc[j]["f_mchapter_nm"]
            if source_chapter != target_chapter:  # 자기 자신 제외
                edges_from_similarity.append((source_chapter, target_chapter))

# 관계 통합 (중복 제거)
final_edges = list(set(edges_from_code + edges_from_similarity))

# 그래프 생성
graph = nx.DiGraph()

# 중단원 노드 추가
unique_mchapters = data["f_mchapter_nm"].unique()
for chapter in unique_mchapters:
    graph.add_node(
        chapter,
        label=chapter,
        color="green",
        size=30
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        source, target,
        color="blue",
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")

# 노드와 엣지를 추가
net.from_nx(graph)

# 물리적 레이아웃 설정 (노드 간 거리 확대)
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -10000,
      "centralGravity": 0.3,
      "springLength": 250,
      "springConstant": 0.05
    },
    "minVelocity": 0.5,
    "solver": "barnesHut"
  },
  "nodes": {
    "shape": "dot",
    "size": 20
  },
  "edges": {
    "smooth": {
      "type": "continuous"
    }
  }
}
""")

# 그래프 저장
output_file = "final_chapter_graph.html"
net.write_html(output_file)
print(f"최적화된 그래프가 '{output_file}'에 저장되었습니다.")


최적화된 그래프가 'final_chapter_graph.html'에 저장되었습니다.


In [61]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# Load your data
file_path = "merged_final_data_new - 복사본.csv" 
data = pd.read_csv(file_path)

# Ensure required columns are present
required_columns = ["f_mchapter_nm", "성취기준코드", "성취기준 내용", "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# Preprocess 중단원명
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\s]", "", regex=True)

# 텍스트 결합
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF 유사도 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 두 가지 유사도 가중합
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 성취기준코드 흐름 기반 관계 생성
edges_from_code = []
previous_chapter = None
for _, row in data.iterrows():
    current_chapter = row["f_mchapter_nm"]
    if previous_chapter and previous_chapter != current_chapter:
        edges_from_code.append((previous_chapter, current_chapter))
    previous_chapter = current_chapter

# 유사도 기반 관계 생성
threshold = 0.8
edges_from_similarity = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):  # 한 방향만 설정
        if final_sim[i, j] >= threshold:
            source_chapter = data.iloc[i]["f_mchapter_nm"]
            target_chapter = data.iloc[j]["f_mchapter_nm"]
            if source_chapter != target_chapter:  # 자기 자신 제외
                edges_from_similarity.append((source_chapter, target_chapter))

# 관계 통합 (중복 제거)
final_edges = list(set(edges_from_code + edges_from_similarity))

# 그래프 생성
graph = nx.DiGraph()

# 중단원 노드 추가
unique_mchapters = data["f_mchapter_nm"].unique()
for chapter in unique_mchapters:
    graph.add_node(
        chapter,
        label=chapter,
        color="green",
        size=30
    )

# 엣지 추가
for source, target in final_edges:
    graph.add_edge(
        source, target,
        color="blue",
        title="선후행 관계 및 유사도 기반 연결"
    )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")

# 노드와 엣지를 추가
net.from_nx(graph)

# 네트워크 데이터를 추출
nodes = [{"id": node, "label": data["label"], "color": data["color"]} for node, data in graph.nodes(data=True)]
edges = [{"from": u, "to": v} for u, v in graph.edges()]

# HTML에 검색 기능 추가
custom_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Knowledge Map</title>
    <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis-network.min.css">
    <style>
        #mynetwork {{
            width: 100%;
            height: 90vh;
            border: 1px solid lightgray;
        }}
        #search-container {{
            margin: 10px;
        }}
    </style>
</head>
<body>
    <div id="search-container">
        <label for="search-input">중단원명 검색:</label>
        <input type="text" id="search-input" placeholder="중단원명을 입력하세요">
        <button onclick="searchNode()">검색</button>
        <button onclick="resetGraph()">전체 보기</button>
    </div>
    <div id="mynetwork"></div>
    <script type="text/javascript">
        var nodes = {nodes};
        var edges = {edges};
        var container = document.getElementById('mynetwork');
        var data = {{ nodes: new vis.DataSet(nodes), edges: new vis.DataSet(edges) }};
        var options = {{
            physics: {{
                barnesHut: {{
                    gravitationalConstant: -10000,
                    centralGravity: 0.3,
                    springLength: 250,
                    springConstant: 0.05
                }},
                minVelocity: 0.5
            }}
        }};
        var network = new vis.Network(container, data, options);

        function searchNode() {{
            var searchValue = document.getElementById('search-input').value.trim();
            if (searchValue === "") {{
                alert("검색할 중단원명을 입력하세요!");
                return;
            }}
            var connectedNodes = network.getConnectedNodes(searchValue);
            if (connectedNodes.length === 0) {{
                alert("해당 중단원명과 연결된 노드가 없습니다.");
                return;
            }}
            var filteredNodes = [searchValue].concat(connectedNodes);
            var newNodes = nodes.filter(node => filteredNodes.includes(node.id));
            var newEdges = edges.filter(edge => filteredNodes.includes(edge.from) && filteredNodes.includes(edge.to));
            network.setData({{ nodes: new vis.DataSet(newNodes), edges: new vis.DataSet(newEdges) }});
        }}

        function resetGraph() {{
            network.setData({{ nodes: new vis.DataSet(nodes), edges: new vis.DataSet(edges) }});
        }}
    </script>
</body>
</html>
"""

# 저장
output_file = "final_chapter_graph_with_search.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(custom_html)

print(f"검색 기능이 추가된 그래프가 '{output_file}'에 저장되었습니다.")


검색 기능이 추가된 그래프가 'final_chapter_graph_with_search.html'에 저장되었습니다.


In [62]:
# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000", notebook=True)

# 노드와 엣지를 추가
net.from_nx(graph)

# 화살표 활성화
for edge in net.edges:
    edge['arrows'] = 'to'  # 화살표가 타겟 방향으로 나타나도록 설정

# HTML에 검색 기능 추가
custom_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Knowledge Map</title>
    <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis-network.min.css">
    <style>
        #mynetwork {{
            width: 100%;
            height: 90vh;
            border: 1px solid lightgray;
        }}
        #search-container {{
            margin: 10px;
        }}
    </style>
</head>
<body>
    <div id="search-container">
        <label for="search-input">중단원명 검색:</label>
        <input type="text" id="search-input" placeholder="중단원명을 입력하세요">
        <button onclick="searchNode()">검색</button>
        <button onclick="resetGraph()">전체 보기</button>
    </div>
    <div id="mynetwork"></div>
    <script type="text/javascript">
        var nodes = {nodes};
        var edges = {edges};
        var container = document.getElementById('mynetwork');
        var data = {{ nodes: new vis.DataSet(nodes), edges: new vis.DataSet(edges) }};
        var options = {{
            physics: {{
                barnesHut: {{
                    gravitationalConstant: -10000,
                    centralGravity: 0.3,
                    springLength: 250,
                    springConstant: 0.05
                }},
                minVelocity: 0.5
            }},
            edges: {{
                arrows: {{
                    to: {{enabled: true}}  // 화살표 방향 표시
                }}
            }}
        }};
        var network = new vis.Network(container, data, options);

        function searchNode() {{
            var searchValue = document.getElementById('search-input').value.trim();
            if (searchValue === "") {{
                alert("검색할 중단원명을 입력하세요!");
                return;
            }}
            var connectedNodes = network.getConnectedNodes(searchValue);
            if (connectedNodes.length === 0) {{
                alert("해당 중단원명과 연결된 노드가 없습니다.");
                return;
            }}
            var filteredNodes = [searchValue].concat(connectedNodes);
            var newNodes = nodes.filter(node => filteredNodes.includes(node.id));
            var newEdges = edges.filter(edge => filteredNodes.includes(edge.from) && filteredNodes.includes(edge.to));
            network.setData({{ nodes: new vis.DataSet(newNodes), edges: new vis.DataSet(newEdges) }});
        }}

        function resetGraph() {{
            network.setData({{ nodes: new vis.DataSet(nodes), edges: new vis.DataSet(edges) }});
        }}
    </script>
</body>
</html>
"""

# 저장
output_file = "final_chapter_graph_with_arrows.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(custom_html)

print(f"화살표가 추가된 그래프가 '{output_file}'에 저장되었습니다.")


화살표가 추가된 그래프가 'final_chapter_graph_with_arrows.html'에 저장되었습니다.


In [82]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
import json

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"
data = pd.read_csv(file_path)

# 필수 컬럼 확인
required_columns = [
    "f_lchapter_nm", "f_mchapter_nm", "f_schapter_nm", "f_tchapter_nm",
    "성취기준코드", "성취기준 내용", "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"필수 컬럼 '{col}'이(가) 데이터에 없습니다.")

# 데이터 전처리
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\s]", "", regex=True)
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 최종 유사도
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가
node_id_map = {}
for index, row in data.iterrows():
    l_id = f"l_{row['f_lchapter_nm']}"
    m_id = f"m_{row['f_mchapter_nm']}"
    s_id = f"s_{row['f_schapter_nm']}"
    t_id = f"t_{row['f_tchapter_nm']}"

    node_id_map[l_id] = row['f_lchapter_nm']
    node_id_map[m_id] = row['f_mchapter_nm']
    node_id_map[s_id] = row['f_schapter_nm']
    node_id_map[t_id] = row['f_tchapter_nm']

    graph.add_node(l_id, label=row['f_lchapter_nm'], color="red", size=40)
    graph.add_node(m_id, label=row['f_mchapter_nm'], color="green", size=30)
    graph.add_node(s_id, label=row['f_schapter_nm'], color="blue", size=20)
    graph.add_node(t_id, label=row['f_tchapter_nm'], color="orange", size=15)

    graph.add_edge(l_id, m_id, color="gray", title="대단원 -> 중단원")
    graph.add_edge(m_id, s_id, color="gray", title="중단원 -> 소단원")
    graph.add_edge(s_id, t_id, color="gray", title="소단원 -> 토픽단원")

# 엣지 추가 (유사도 기반)
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= 0.75:
            source = f"m_{data.iloc[i]['f_mchapter_nm']}"
            target = f"m_{data.iloc[j]['f_mchapter_nm']}"
            graph.add_edge(source, target, color="blue", title="유사도 기반 연결")

# PyVis 생성
net = Network(height="90vh", width="100%", directed=True)
net.from_nx(graph)

# HTML 생성
nodes = json.dumps(net.get_nodes())
edges = json.dumps(net.get_edges())

custom_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Interactive Knowledge Map</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"></script>
    <style>
        #mynetwork {{ width: 100%; height: 90vh; border: 1px solid lightgray; }}
    </style>
</head>
<body>
    <div id="mynetwork"></div>
    <script>
        var nodes = {nodes};
        var edges = {edges};
        var container = document.getElementById('mynetwork');
        var data = {{ nodes: new vis.DataSet(nodes), edges: new vis.DataSet(edges) }};
        var options = {{
            physics: {{
                barnesHut: {{
                    gravitationalConstant: -2000,
                    centralGravity: 0.5,
                    springLength: 100
                }}
            }},
            interaction: {{ hover: true }}
        }};
        var network = new vis.Network(container, data, options);
    </script>
</body>
</html>
"""

# HTML 저장
output_file = "interactive_knowledge_map.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(custom_html)

print(f"Interactive knowledge map saved to {output_file}")


Interactive knowledge map saved to interactive_knowledge_map.html


In [85]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network
import json

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"
data = pd.read_csv(file_path)

# NaN 값 제거
data = data.dropna(subset=["f_lchapter_nm", "f_mchapter_nm", "f_schapter_nm", "f_tchapter_nm"])

# 유사도 계산 준비
data["bert_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["tfidf_text"] = data["핵심키워드_v2"].fillna("")

# 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(data["bert_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["tfidf_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 최종 유사도
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가
for _, row in data.iterrows():
    l_id = f"l_{row['f_lchapter_nm']}"
    m_id = f"m_{row['f_mchapter_nm']}"
    s_id = f"s_{row['f_schapter_nm']}"
    t_id = f"t_{row['f_tchapter_nm']}"

    graph.add_node(l_id, label=row['f_lchapter_nm'], color="red", size=40)
    graph.add_node(m_id, label=row['f_mchapter_nm'], color="green", size=30)
    graph.add_node(s_id, label=row['f_schapter_nm'], color="blue", size=20)
    graph.add_node(t_id, label=row['f_tchapter_nm'], color="orange", size=15)

    graph.add_edge(l_id, m_id, color="gray", title="대단원 -> 중단원")
    graph.add_edge(m_id, s_id, color="gray", title="중단원 -> 소단원")
    graph.add_edge(s_id, t_id, color="gray", title="소단원 -> 토픽단원")

# 유사도 기반 엣지 추가
threshold = 0.75
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if final_sim[i, j] >= threshold:
            source = f"m_{data.iloc[i]['f_mchapter_nm']}"
            target = f"m_{data.iloc[j]['f_mchapter_nm']}"
            graph.add_edge(source, target, color="blue", title="유사도 기반 연결")

# PyVis 생성
net = Network(height="90vh", width="100%", directed=True)
net.from_nx(graph)

# JSON 데이터 생성
nodes = json.dumps(net.get_nodes())
edges = json.dumps(net.get_edges())

# HTML 저장
custom_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Interactive Knowledge Map</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/vis/4.21.0/vis.min.js"></script>
    <style>#mynetwork {{ width: 100%; height: 90vh; }}</style>
</head>
<body>
    <div id="mynetwork"></div>
    <script>
        var nodes = {nodes};
        var edges = {edges};
        var container = document.getElementById('mynetwork');
        var data = {{ nodes: new vis.DataSet(nodes), edges: new vis.DataSet(edges) }};
        var options = {{
            physics: {{
                barnesHut: {{
                    gravitationalConstant: -2000,
                    centralGravity: 0.3,
                    springLength: 150
                }}
            }}
        }};
        var network = new vis.Network(container, data, options);
    </script>
</body>
</html>
"""

output_file = "interactive_knowledge_map.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(custom_html)

print(f"Interactive knowledge map saved to {output_file}")


Interactive knowledge map saved to interactive_knowledge_map.html


In [38]:
# 데이터에서 고유 중단원명 확인
unique_mchapters = set(data["f_mchapter_nm"].unique())

# 그래프에서 사용된 노드 확인
graph_nodes = set(graph.nodes)

# 빠진 중단원명 확인
missing_mchapters = unique_mchapters - graph_nodes
print(f"빠진 중단원명: {missing_mchapters}")


빠진 중단원명: set()


In [39]:
# 모든 엣지가 한 방향인지 확인
direction_errors = [
    (source, target) for source, target in graph.edges
    if graph.has_edge(target, source)  # 역방향 엣지가 있는지 확인
]

print(f"양방향 엣지: {direction_errors}")


양방향 엣지: [('두 수의 크기 비교', '여러 개의 수의 크기 비교하기'), ('두 수의 크기 비교', '1000과 몇천 알아보기'), ('두 수의 크기 비교', '수의 순서 알아보기'), ('두 수의 크기 비교', '네 자리 수 알아보기'), ('두 수의 크기 비교', '두 수의 크기 비교하기'), ('두 수의 크기 비교', '뛰어 세기'), ('두 수의 크기 비교', '백과 몇백 알아보기'), ('여러 가지 모양 찾아보기', '여러 가지 모양 알아보기'), ('여러 가지 모양 찾아보기', '도형'), ('여러 가지 모양 찾아보기', '쌓기나무'), ('여러 가지 모양 알아보기', '여러 가지 모양 찾아보기'), ('여러 가지 모양 알아보기', '도형'), ('여러 가지 모양 알아보기', '쌓기나무'), ('덧셈', '뺄셈'), ('뺄셈', '덧셈'), ('길이 비교하기', '담을 수 있는 양 비교하기'), ('길이 비교하기', '높이 비교하기'), ('길이 비교하기', '키 비교하기'), ('길이 비교하기', '무게 비교하기'), ('길이 비교하기', '넓이 비교하기'), ('무게 비교하기', '길이 비교하기'), ('넓이 비교하기', '길이 비교하기'), ('담을 수 있는 양 비교하기', '길이 비교하기'), ('높이 비교하기', '길이 비교하기'), ('키 비교하기', '길이 비교하기'), ('백과 몇백 알아보기', '두 수의 크기 비교'), ('뛰어 세기', '네 자리 수 알아보기'), ('뛰어 세기', '두 수의 크기 비교하기'), ('뛰어 세기', '수의 순서 알아보기'), ('뛰어 세기', '여러 개의 수의 크기 비교하기'), ('뛰어 세기', '두 수의 크기 비교'), ('뛰어 세기', '1000과 몇천 알아보기'), ('도형', '여러 가지 모양 찾아보기'), ('도형', '여러 가지 모양 알아보기'), ('쌓기나무', '여러 가지 모양 알아보기'), ('쌓기나무', '여러 가지 모양 찾아보기'), ('수의 순서 알아보기', '두 수의 크

In [41]:
# 중복된 성취기준코드 확인
duplicated_codes = data[data.duplicated(subset=["성취기준코드"], keep=False)]
print(duplicated_codes)

# 중복된 중단원 확인
duplicated_chapters = data[data.duplicated(subset=["f_mchapter_nm"], keep=False)]
print(duplicated_chapters)


     f_subject_id  f_lchapter_id f_lchapter_nm  f_mchapter_id   f_mchapter_nm  \
0            2212       17122834     1. 9까지의 수       14201779  1 2 3 4 5 알아보기   
1            2212       17122834     1. 9까지의 수       14201779  1 2 3 4 5 알아보기   
2            2212       17122834     1. 9까지의 수       14201779  1 2 3 4 5 알아보기   
3            2212       17122834     1. 9까지의 수       14201779  1 2 3 4 5 알아보기   
4            2212       17122834     1. 9까지의 수       14201779  1 2 3 4 5 알아보기   
..            ...            ...           ...            ...             ...   
799          2215       17122923      6. 규칙 찾기       14201907           규칙 찾기   
800          2215       17122923      6. 규칙 찾기       14201907           규칙 찾기   
801          2215       17122923      6. 규칙 찾기       14201907           규칙 찾기   
802          2215       17122923      6. 규칙 찾기       14201907           규칙 찾기   
803          2215       17122923      6. 규칙 찾기       14201907           규칙 찾기   

      성취기준코드               

In [42]:
# 양방향 관계로 나타난 노드의 성취기준코드 흐름 확인
for source, target in direction_errors:
    print(data[(data["성취기준코드"] == source) | (data["성취기준코드"] == target)])


Empty DataFrame
Columns: [f_subject_id, f_lchapter_id, f_lchapter_nm, f_mchapter_id, f_mchapter_nm, 성취기준코드, 성취기준 내용, 성취수준 A, 성취수준 B, 성취수준 C, f_schapter_id, f_schapter_nm, f_tchapter_id, f_tchapter_nm, 핵심키워드_v2, 핵심키워드_v1, bert_text, tfidf_text]
Index: []
Empty DataFrame
Columns: [f_subject_id, f_lchapter_id, f_lchapter_nm, f_mchapter_id, f_mchapter_nm, 성취기준코드, 성취기준 내용, 성취수준 A, 성취수준 B, 성취수준 C, f_schapter_id, f_schapter_nm, f_tchapter_id, f_tchapter_nm, 핵심키워드_v2, 핵심키워드_v1, bert_text, tfidf_text]
Index: []
Empty DataFrame
Columns: [f_subject_id, f_lchapter_id, f_lchapter_nm, f_mchapter_id, f_mchapter_nm, 성취기준코드, 성취기준 내용, 성취수준 A, 성취수준 B, 성취수준 C, f_schapter_id, f_schapter_nm, f_tchapter_id, f_tchapter_nm, 핵심키워드_v2, 핵심키워드_v1, bert_text, tfidf_text]
Index: []
Empty DataFrame
Columns: [f_subject_id, f_lchapter_id, f_lchapter_nm, f_mchapter_id, f_mchapter_nm, 성취기준코드, 성취기준 내용, 성취수준 A, 성취수준 B, 성취수준 C, f_schapter_id, f_schapter_nm, f_tchapter_id, f_tchapter_nm, 핵심키워드_v2, 핵심키워드_v1, bert_text, tfidf_

In [44]:
# source와 target 값이 데이터에 존재하는지 확인
for source, target in direction_errors:
    if source not in data["성취기준코드"].values or target not in data["성취기준코드"].values:
        print(f"성취기준코드 {source} 또는 {target}이 데이터에 존재하지 않습니다.")

성취기준코드 두 수의 크기 비교 또는 여러 개의 수의 크기 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 두 수의 크기 비교 또는 1000과 몇천 알아보기이 데이터에 존재하지 않습니다.
성취기준코드 두 수의 크기 비교 또는 수의 순서 알아보기이 데이터에 존재하지 않습니다.
성취기준코드 두 수의 크기 비교 또는 네 자리 수 알아보기이 데이터에 존재하지 않습니다.
성취기준코드 두 수의 크기 비교 또는 두 수의 크기 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 두 수의 크기 비교 또는 뛰어 세기이 데이터에 존재하지 않습니다.
성취기준코드 두 수의 크기 비교 또는 백과 몇백 알아보기이 데이터에 존재하지 않습니다.
성취기준코드 여러 가지 모양 찾아보기 또는 여러 가지 모양 알아보기이 데이터에 존재하지 않습니다.
성취기준코드 여러 가지 모양 찾아보기 또는 도형이 데이터에 존재하지 않습니다.
성취기준코드 여러 가지 모양 찾아보기 또는 쌓기나무이 데이터에 존재하지 않습니다.
성취기준코드 여러 가지 모양 알아보기 또는 여러 가지 모양 찾아보기이 데이터에 존재하지 않습니다.
성취기준코드 여러 가지 모양 알아보기 또는 도형이 데이터에 존재하지 않습니다.
성취기준코드 여러 가지 모양 알아보기 또는 쌓기나무이 데이터에 존재하지 않습니다.
성취기준코드 덧셈 또는 뺄셈이 데이터에 존재하지 않습니다.
성취기준코드 뺄셈 또는 덧셈이 데이터에 존재하지 않습니다.
성취기준코드 길이 비교하기 또는 담을 수 있는 양 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 길이 비교하기 또는 높이 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 길이 비교하기 또는 키 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 길이 비교하기 또는 무게 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 길이 비교하기 또는 넓이 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 무게 비교하기 또는 길이 비교하기이 데이터에 존재하지 않습니다.
성취기준코드 넓이 비교하기 또는 길이 비교하기이 데이터

In [45]:
# 성취기준코드 열 데이터 타입 확인
print(data["성취기준코드"].dtype)

# direction_errors 값 타입 확인
print(type(direction_errors[0][0]))


object
<class 'str'>


### 위의 코드들로 오류 파악 후 수정한 코드

In [104]:
import pandas as pd
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"  # 파일 경로
data = pd.read_csv(file_path)

# 필수 컬럼 확인
required_columns = ["f_mchapter_nm", "성취기준코드", "성취기준 내용", "핵심키워드_v2", "성취수준 A", "성취수준 B", "성취수준 C"]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"필수 컬럼 '{col}'이(가) 데이터에 없습니다.")

# 텍스트 정제
data["f_mchapter_nm"] = data["f_mchapter_nm"].str.strip().str.replace(r"[^가-힣a-zA-Z0-9\s]", "", regex=True)
data["combined_text"] = (
    data["성취기준 내용"].fillna("") + " " +
    data["성취수준 A"].fillna("") + " " +
    data["성취수준 B"].fillna("") + " " +
    data["성취수준 C"].fillna("")
)
data["keyword_text"] = data["핵심키워드_v2"].fillna("")

# Sentence-BERT 유사도 계산
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = bert_model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(bert_embeddings.cpu(), bert_embeddings.cpu())

# TF-IDF 유사도 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["keyword_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 최종 유사도 계산 (가중치 적용)
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim
similarity_threshold = 0.75

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가 (중단원)
unique_mchapters = data["f_mchapter_nm"].drop_duplicates().tolist()
for chapter in unique_mchapters:
    graph.add_node(
        chapter,
        label=chapter,
        color="green",
        size=30
    )

# 유사도 기반 엣지 추가
for i in range(len(data)):
    for j in range(i + 1, len(data)):  # 한 방향으로만 연결
        if final_sim[i, j] >= similarity_threshold:
            graph.add_edge(
                data.iloc[i]["f_mchapter_nm"],
                data.iloc[j]["f_mchapter_nm"],
                weight=final_sim[i, j],
                color="blue",
                title=f"유사도: {final_sim[i, j]:.2f}"
            )

# 성취기준 흐름 기반 엣지 추가
previous_chapter = None
for _, row in data.iterrows():
    current_chapter = row["f_mchapter_nm"]
    if previous_chapter and previous_chapter != current_chapter:
        graph.add_edge(
            previous_chapter, current_chapter,
            weight=1.0,
            color="gray",
            title="성취기준 흐름 기반 연결"
        )
    previous_chapter = current_chapter

# 자기 자신을 가리키는 루프 제거
graph.remove_edges_from(nx.selfloop_edges(graph))

# PyVis 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Physics 설정
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -8000,
      "centralGravity": 0.3,
      "springLength": 200,
      "springConstant": 0.05
    },
    "minVelocity": 0.75
  }
}
""")

# HTML 저장
output_file = "final_chapter_graph2.html"
net.write_html(output_file)
print(f"Knowledge map saved to {output_file}")


Knowledge map saved to final_chapter_graph2.html


### 유사도 검사 + 성취기준 코드 

### f_mchapter_id / f_mchapter_nm 고유값 각 94개, 84개

In [3]:
# 고유 값 확인
unique_values = data["f_mchapter_id"].unique()  # 중복 제거한 고유 값
print("고유 중단원명:", unique_values)

# 고유 값 개수
unique_count = len(unique_values)
print(f"고유 중단원명의 개수: {unique_count}")

고유 중단원명: [14201779 14201799 14201798 14201889 14201857 14201858 14201780 14201803
 14201800 14201804 14201890 14201892 14201802 14201859 14201891 14201781
 14201860 14201801 14201861 14201806 14201805 14201786 14201874 14201787
 14201862 14201790 14201867 14201782 14201866 14201881 14201888 14201884
 14201885 14201810 14201864 14201863 14201789 14201788 14201865 14201886
 14201882 14201875 14201876 14201809 14201783 14201791 14201811 14201887
 14201883 14201813 14201812 14201821 14201894 14201820 14201819 14201893
 14201896 14201897 14201895 14201907 14201880 14201877 14201878 14201879
 14201784 14201808 14201807 14201785 14201868 14201869 14201870 14201794
 14201793 14201795 14201797 14201796 14201792 14201814 14201902 14201871
 14201872 14201903 14201904 14201873 14201815 14201898 14201816 14201817
 14201901 14201899 14201900 14201818 14201905 14201906]
고유 중단원명의 개수: 94


In [28]:
import pandas as pd

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# f_mchapter_id 중복 제거 후 매핑된 f_mchapter_nm, f_lchapter_id, f_lchapter_nm 가져오기
unique_data = data.drop_duplicates(subset=["f_mchapter_id"])[
    ["f_lchapter_id", "f_lchapter_nm", "f_mchapter_id", "f_mchapter_nm"]
]

# 결과 출력
print("f_mchapter_id 고유값과 매핑된 f_mchapter_nm, 대단원 정보:")
print(unique_data)

# 파일로 저장
output_file = "unique_f_mchapter_id_with_lchapter_mapping.csv"
unique_data.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"파일로 저장되었습니다: {output_file}")


f_mchapter_id 고유값과 매핑된 f_mchapter_nm, 대단원 정보:
     f_lchapter_id f_lchapter_nm  f_mchapter_id                  f_mchapter_nm
0         17122834     1. 9까지의 수       14201779             1, 2, 3, 4, 5 알아보기
6         17122834     1. 9까지의 수       14201780                6, 7, 8, 9 알아보기
11        17122834     1. 9까지의 수       14201781                      9까지 수의 순서
17        17122834     1. 9까지의 수       14201782  1만큼 더 큰 수와 1만큼 더 작은 수, 0 알아보기
25        17122834     1. 9까지의 수       14201783                     두 수의 크기 비교
..             ...           ...            ...                            ...
759       17122921     4. 시각과 시간       14201903                    하루의 시간 알아보기
766       17122921     4. 시각과 시간       14201904                        달력 알아보기
773       17122922     5. 표와 그래프       14201905                        표로 나타내기
782       17122922     5. 표와 그래프       14201906                      그래프로 나타내기
791       17122923      6. 규칙 찾기       14201907                          규칙 찾기

[94 r

In [32]:
import pandas as pd

# 데이터 로드
file_path = "unique_f_mchapter_id_with_lchapter_mapping.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# 중복된 f_mchapter_nm 추출
duplicate_names = data[data.duplicated(subset=["f_mchapter_nm"], keep=False)]

# 고유한 f_mchapter_id, f_mchapter_nm, f_lchapter_id, f_lchapter_nm 추출
unique_duplicates = duplicate_names[["f_mchapter_id", "f_mchapter_nm", "f_lchapter_id", "f_lchapter_nm"]].drop_duplicates()

# 결과 출력
print("중복된 f_mchapter_nm에 대한 고유한 매핑:")
print(unique_duplicates)

# 파일로 저장
output_file = "duplicate_f_mchapter_nm_with_lchapter_mapping.xlsx"
unique_duplicates.to_excel(output_file)
print(f"결과가 파일로 저장되었습니다: {output_file}")


중복된 f_mchapter_nm에 대한 고유한 매핑:
    f_mchapter_id  f_mchapter_nm  f_lchapter_id f_lchapter_nm
4        14201783     두 수의 크기 비교       17122834     1. 9까지의 수
5        14201784  여러 가지 모양 찾아보기       17122835   2. 여러 가지 모양
6        14201785  여러 가지 모양 알아보기       17122835   2. 여러 가지 모양
10       14201789             덧셈       17122836     3. 덧셈과 뺄셈
11       14201790             뺄셈       17122836     3. 덧셈과 뺄셈
13       14201792        길이 비교하기       17122837       4. 비교하기
23       14201802     두 수의 크기 비교       17122838    5. 50까지의 수
26       14201805          뛰어 세기       17122839     1. 세 자리 수
27       14201806     두 수의 크기 비교       17122839     1. 세 자리 수
30       14201809             덧셈       17122841     3. 덧셈과 뺄셈
31       14201810             뺄셈       17122841     3. 덧셈과 뺄셈
35       14201814        길이 비교하기       17122842      4. 길이 재기
54       14201868  여러 가지 모양 찾아보기       17122914     3. 모양과 시각
55       14201869  여러 가지 모양 알아보기       17122914     3. 모양과 시각
63       14201877          규칙 찾기       1


### 소단원까지!

In [62]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pyvis.network import Network

# 데이터 로드
file_path = "merged_final_data_new - 복사본.csv"  # 데이터 파일 경로
data = pd.read_csv(file_path)

# 필수 컬럼 확인
required_columns = [
    "f_mchapter_nm", "f_mchapter_id", "성취기준코드", "성취기준 내용",
    "성취수준 A", "성취수준 B", "성취수준 C", "f_schapter_id",
    "f_schapter_nm", "핵심키워드_v2"
]
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Required column '{col}' is missing from the data")

# 데이터 전처리
data = data.drop_duplicates(subset=["f_mchapter_nm", "성취기준 내용", "핵심키워드_v2"])
data["combined_text"] = (
    data["f_mchapter_nm"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["성취기준 내용"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True) +
    " " +
    data["핵심키워드_v2"].str.lower().str.replace(r"[^a-z가-힣0-9\s]", "", regex=True)
)
data["tooltip"] = (
    "소단원명: " + data["f_schapter_nm"].fillna("N/A") +
    "<br>성취수준 A: " + data["성취수준 A"].fillna("") +
    "<br>성취수준 B: " + data["성취수준 B"].fillna("") +
    "<br>성취수준 C: " + data["성취수준 C"].fillna("")
)

# 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(data["combined_text"].tolist(), convert_to_tensor=True)
bert_sim = cosine_similarity(embeddings.cpu(), embeddings.cpu())

# TF-IDF 유사도 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data["combined_text"])
tfidf_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 최종 유사도 계산
final_sim = 0.8 * bert_sim + 0.2 * tfidf_sim
threshold = 0.75

# 그래프 생성
graph = nx.DiGraph()

# 노드 추가 (소단원 포함)
for _, row in data.iterrows():
    # 성취기준코드 노드
    graph.add_node(
        row["성취기준코드"],
        label=row["f_schapter_nm"] + " - " + row["성취기준코드"],
        tooltip=row["tooltip"],
        color="red",
        size=20
    )
    # 소단원명 노드
    if pd.notna(row["f_schapter_id"]):
        graph.add_node(
            row["f_schapter_id"],
            label=row["f_schapter_nm"],
            tooltip="소단원",
            color="lightblue",
            size=15
        )
        # 소단원과 성취기준 연결
        graph.add_edge(
            row["f_schapter_id"],
            row["성취기준코드"],
            weight=1.0,
            title="소단원 연결"
        )

# 선후행 관계 정의
predefined_edges = [
    ("2수01-01", "2수01-02"),
    ("2수01-02", "2수01-03"),
    ("2수01-03", "2수01-04"),
    ("2수01-05", "2수01-06"),
    ("2수01-06", "2수01-07"),
    ("2수01-07", "2수01-08"),
    ("2수01-08", "2수01-09"),
    ("2수01-10", "2수01-11"),
    ("2수02-01", "2수02-02"),
    ("2수03-01", "2수03-02"),
    ("2수03-03", "2수03-04"),
    ("2수03-04", "2수03-05"),
    ("2수03-06", "2수03-07"),
    ("2수03-07", "2수03-08"),
    ("2수03-08", "2수03-09"),
    ("2수03-10", "2수03-11"),
    ("2수03-11", "2수03-12"),
    ("2수03-12", "2수03-13"),
    ("2수04-01", "2수04-02"),
    ("2수04-02", "2수04-03")
]
for edge in predefined_edges:
    graph.add_edge(edge[0], edge[1], weight=1.0, title="선후행 관계")

# 유사도 기반 엣지 추가
for i in range(len(data)):
    for j in range(len(data)):
        if i != j and final_sim[i, j] >= threshold:
            source = data.iloc[i]["성취기준코드"]
            target = data.iloc[j]["성취기준코드"]
            if not graph.has_edge(source, target):
                graph.add_edge(
                    source,
                    target,
                    weight=final_sim[i, j],
                    title=f"유사도: {final_sim[i, j]:.2f}",
                    color="blue",
                    dash=True
                )

# PyVis로 시각화
net = Network(height="100vh", width="100vw", directed=True, bgcolor="#ffffff", font_color="#000000")
net.from_nx(graph)

# Physics 설정
net.set_options("""
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000,
      "centralGravity": 0.4,
      "springLength": 300,
      "springConstant": 0.1
    },
    "minVelocity": 0.5
  }
}
""")

# 시각화 저장
output_file = "knowledge_graph_with_subchapter.html"
net.write_html(output_file)
print(f"Graph saved to {output_file}")


Graph saved to knowledge_graph_with_subchapter.html
